In [2]:
# === CNT one-shot environment + GPU check (single cell) ===
import sys, subprocess, platform, json, shutil

def run(cmd):
    try:
        r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, check=False)
        return r.returncode, r.stdout.strip()
    except Exception as e:
        return -1, str(e)

def pip_install(args):
    return run([sys.executable, "-m", "pip", "install"] + args)

print("== Kernel info ==")
print("Python:", platform.python_version())
print("Interpreter:", sys.executable)

print("\n== Upgrading pip core ==")
print(pip_install(["--upgrade", "pip", "setuptools", "wheel"])[1])

# --- Core scientific stack (CPU-safe) ---
core = ["numpy","scipy","pandas","matplotlib","statsmodels","scikit-learn","numba","umap-learn","networkx"]
viz  = ["plotly","kaleido"]
forecast = ["statsforecast","pmdarima"]
neuro_genomics = ["mne","nilearn","anndata","scanpy","pybedtools","pybigwig"]
maybe_heavy = ["ubermag"]  # will set up OOMMF on first real use

print("\n== Installing core scientific stack ==")
print(pip_install(core + viz + forecast + neuro_genomics + maybe_heavy)[1])

# --- Torch GPU (CUDA 12.4 wheels) with graceful fallback ---
torch_gpu_ok = False
print("\n== Installing PyTorch (CUDA 12.4) → fallback to CPU if needed ==")
code, out = pip_install(["--index-url","https://download.pytorch.org/whl/cu124","torch","torchvision","torchaudio"])
if code != 0:
    print("[torch cuda install failed] falling back to CPU wheels…")
    print(pip_install(["torch","torchvision","torchaudio"])[1])
else:
    print(out)

# --- TensorFlow (GPU if drivers/toolkit match) ---
print("\n== Installing TensorFlow (may be CPU if no compatible GPU build) ==")
print(pip_install(["tensorflow"])[1])

# --- Sanity imports & versions ---
summary = {"torch":None,"cuda_available":None,"cuda_device":None,"tf":None,"tf_gpus":None,"nvidia_smi":None}

print("\n== Import checks ==")
try:
    import torch
    summary["torch"] = getattr(torch, "__version__", "unknown")
    summary["cuda_available"] = bool(torch.cuda.is_available())
    summary["cuda_device"] = (torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
    print(f"Torch: {summary['torch']}  | CUDA available: {summary['cuda_available']}  | Device: {summary['cuda_device']}")
except Exception as e:
    print("Torch import failed:", e)

try:
    import tensorflow as tf
    summary["tf"] = getattr(tf, "__version__", "unknown")
    gpus = tf.config.list_physical_devices('GPU')
    summary["tf_gpus"] = [g.name for g in gpus] if gpus else []
    print(f"TensorFlow: {summary['tf']}  | GPUs: {summary['tf_gpus']}")
except Exception as e:
    print("TensorFlow import failed:", e)

# --- OS-level GPU probe (nvidia-smi) ---
print("\n== nvidia-smi probe ==")
code, out = run(["nvidia-smi"])
summary["nvidia_smi"] = (out if code == 0 else "nvidia-smi not found or no NVIDIA driver.")
print(out)

# --- Quick GPU spike tests (safe sizes) ---
print("\n== Quick GPU spike tests ==")
try:
    import torch, time
    if torch.cuda.is_available():
        x = torch.randn(4096, 4096, device="cuda")
        t0 = time.time(); y = x @ x; torch.cuda.synchronize(); dt = time.time()-t0
        print(f"PyTorch CUDA matmul OK in {dt:.3f}s, y.sum()={float(y.sum()):.3e}")
    else:
        print("PyTorch: CUDA not available, skipping GPU matmul.")
except Exception as e:
    print("PyTorch spike failed:", e)

try:
    import tensorflow as tf, time
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        with tf.device('/GPU:0'):
            a = tf.random.normal((4096,4096))
            t0 = time.time(); b = a @ a; _ = tf.reduce_sum(b).numpy(); dt = time.time()-t0
            print(f"TensorFlow GPU matmul OK in {dt:.3f}s")
    else:
        print("TensorFlow: no GPU visible, skipping GPU matmul.")
except Exception as e:
    print("TensorFlow spike failed:", e)

# --- Final print ---
print("\n== CNT environment summary ==")
print(json.dumps(summary, indent=2))
print("\nAll set. If the kernel was just updated with new packages, consider doing Kernel → Restart once.")


== Kernel info ==
Python: 3.13.5
Interpreter: C:\Users\caleb\cnt_genome\.venv\Scripts\python.exe

== Upgrading pip core ==

== Installing core scientific stack ==
  Using cached kaleido-1.1.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached statsforecast-2.0.2.tar.gz (2.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pmdarima-2.0.4.tar.gz (630 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached n

KeyboardInterrupt: 

In [ ]:
# === CNT Grand Notebook: Initialization ===
!pip install numpy pandas matplotlib scipy scikit-learn statsmodels umap-learn plotly pywavelets tqdm --quiet

import os, sys, platform, json, time, random, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import stats, signal
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")

print("Python:", platform.python_version())
print("GPU available:", "✅" if os.system("nvidia-smi >nul 2>&1")==0 else "❌")
print("Initialization complete.")


In [ ]:
# === CNT Core Engine ===
def cnt_resonance_analysis(X, name="CNT_Test"):
    """
    Performs correlation resonance mapping, entropy drift, and coherence collapse tests.
    Input: X (2D numpy array)
    """
    X = StandardScaler().fit_transform(X)
    corr = np.corrcoef(X.T)
    eigvals, eigvecs = np.linalg.eig(corr)
    entropy = -np.sum((eigvals / np.sum(eigvals)) * np.log(np.abs(eigvals / np.sum(eigvals))))

    plt.figure(figsize=(6,5))
    plt.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar(label='Correlation')
    plt.title(f"Resonance Map — {name}\nEntropy={entropy:.3f}")
    plt.show()
    return {"entropy": entropy, "eigvals": eigvals, "corr": corr}


In [ ]:
# === CNT Ground Test ===
np.random.seed(42)
T, N = 1000, 8
signals = np.array([np.sin(np.linspace(0, 10*np.pi, T) + np.random.rand()*5) + np.random.randn(T)*0.1 for _ in range(N)]).T
result = cnt_resonance_analysis(signals, "Synthetic Cognitive Field")

print("Entropy:", result["entropy"])


In [ ]:
# === CNT Resonance Entropy Law Validation ===
import numpy as np, matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

def cnt_entropy(X):
    X = StandardScaler().fit_transform(X)
    corr = np.corrcoef(X.T)
    eigvals, _ = np.linalg.eig(corr)
    eigvals = np.abs(eigvals) / np.sum(np.abs(eigvals))
    return -np.sum(eigvals * np.log(eigvals + 1e-12))

def cnt_entropy_permutation_test(X, n_perm=2000, show_plot=True):
    """Permutation test for CNT Resonance Entropy Law."""
    obs = cnt_entropy(X)
    null_ent = np.zeros(n_perm)
    flat = X.flatten()

    for i in tqdm(range(n_perm), desc="Permuting nulls"):
        np.random.shuffle(flat)
        Xp = flat.reshape(X.shape)
        null_ent[i] = cnt_entropy(Xp)

    p = np.mean(null_ent <= obs)

    if show_plot:
        plt.figure(figsize=(7,4))
        plt.hist(null_ent, bins=50, color='lightgray', edgecolor='k', alpha=0.8)
        plt.axvline(obs, color='red', lw=2, label=f"Observed = {obs:.3f}")
        plt.title(f"CNT Resonance Entropy Law Validation\np = {p:.5f}")
        plt.xlabel("Entropy under Null")
        plt.ylabel("Frequency")
        plt.legend()
        plt.show()

    return {"observed_entropy": obs, "p_value": p, "null_distribution": null_ent}

# === Run the test on your current field ===
np.random.seed(42)
T, N = 1000, 8
signals = np.array([
    np.sin(np.linspace(0, 10*np.pi, T) + np.random.rand()*5) + np.random.randn(T)*0.1
    for _ in range(N)
]).T

res = cnt_entropy_permutation_test(signals, n_perm=2000)
print(f"Observed Entropy = {res['observed_entropy']:.4f}, p = {res['p_value']:.5f}")


In [ ]:
# === CNT Collapse-Field Framework ===
import numpy as np, matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

def cnt_entropy(X):
    X = StandardScaler().fit_transform(X)
    corr = np.corrcoef(X.T)
    eigvals, _ = np.linalg.eig(corr)
    eigvals = np.abs(eigvals) / np.sum(np.abs(eigvals))
    return -np.sum(eigvals * np.log(eigvals + 1e-12))

def cnt_apply_perturbation(X, intensity):
    """Inject Gaussian and structural noise at a given intensity."""
    noise = np.random.randn(*X.shape) * intensity
    shuffled = X.copy()
    for _ in range(int(intensity * 10)):      # random re-ordering proportional to intensity
        i, j = np.random.randint(0, X.shape[0], 2)
        shuffled[[i, j]] = shuffled[[j, i]]
    return X + noise + 0.0 * shuffled  # structural + additive drift

def cnt_collapse_test(X, steps=np.linspace(0,1,25)):
    """Quantify entropy drift under progressive perturbation."""
    base_entropy = cnt_entropy(X)
    entropies = []
    for s in tqdm(steps, desc="Inducing collapse"):
        entropies.append(cnt_entropy(cnt_apply_perturbation(X, s)))
    entropies = np.array(entropies)
    collapse_idx = np.argmax(entropies > (base_entropy + 0.25))  # collapse threshold
    collapse_intensity = steps[collapse_idx] if collapse_idx>0 else np.nan

    plt.figure(figsize=(7,4))
    plt.plot(steps, entropies, 'o-', label="Entropy under drift")
    plt.axhline(base_entropy, color='red', linestyle='--', label=f"Base Entropy={base_entropy:.3f}")
    plt.xlabel("Perturbation Intensity")
    plt.ylabel("Entropy")
    plt.title("CNT Collapse Field Simulation")
    plt.legend()
    plt.show()

    return {"base_entropy": base_entropy,
            "entropies": entropies,
            "collapse_intensity": collapse_intensity}

# === Run Collapse Simulation ===
np.random.seed(42)
T, N = 1000, 8
signals = np.array([
    np.sin(np.linspace(0, 10*np.pi, T) + np.random.rand()*5) + np.random.randn(T)*0.1
    for _ in range(N)
]).T

collapse = cnt_collapse_test(signals)
print(f"Base Entropy = {collapse['base_entropy']:.3f}")
print(f"Collapse Intensity ≈ {collapse['collapse_intensity']}")


In [ ]:
# === CNT Phase Equation Fit & Visualization ===
import numpy as np, matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# Re-use your entropy and perturbation arrays from the collapse run
steps = np.linspace(0, 1, 25)
entropies = collapse["entropies"]
S0 = collapse["base_entropy"]
gamma_c = collapse["collapse_intensity"]
Δc = 0.25

# --- Theoretical CNT Phase Equation ---
def S_theoretical(gamma, k_r, beta):
    return S0 + k_r * np.exp(beta * (gamma - gamma_c))

# --- Fit coefficients to observed data ---
popt, pcov = curve_fit(S_theoretical, steps, entropies, p0=[0.25, 4.0])
k_r, beta = popt
stderr = np.sqrt(np.diag(pcov))

# --- Generate smooth theoretical curve ---
gamma_fit = np.linspace(0, 1, 200)
S_fit = S_theoretical(gamma_fit, *popt)

# --- Plot ---
plt.figure(figsize=(8,5))
plt.plot(steps, entropies, "o", label="Observed Entropy", alpha=0.8)
plt.plot(gamma_fit, S_fit, "-", lw=2.5, label=f"Theoretical Fit  (k_r={k_r:.3f}, β={beta:.3f})")
plt.axvline(gamma_c, color="red", ls="--", lw=1.5, label=f"Collapse Threshold γc={gamma_c:.2f}")
plt.xlabel("Perturbation Intensity γ")
plt.ylabel("Entropy  S(γ)")
plt.title("CNT Phase Equation Fit — Resonance→Collapse Continuum")
plt.legend()
plt.grid(alpha=0.25)
plt.show()

# --- Summary of Stability Coefficients ---
R_n = 1 - gamma_c
print("=== CNT Phase Equation Coefficients ===")
print(f"k_r (Resonance rate constant): {k_r:.5f} ± {stderr[0]:.5f}")
print(f"β  (Field sensitivity):         {beta:.5f} ± {stderr[1]:.5f}")
print(f"γ_c (Collapse threshold):       {gamma_c:.3f}")
print(f"Rₙ  (Resilience Index):         {R_n:.3f}")


In [ ]:
# === CNT Reconstruction Simulation ===
import numpy as np, matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

def cnt_entropy(X):
    X = StandardScaler().fit_transform(X)
    corr = np.corrcoef(X.T)
    eigvals, _ = np.linalg.eig(corr)
    eigvals = np.abs(eigvals) / np.sum(np.abs(eigvals))
    return -np.sum(eigvals * np.log(eigvals + 1e-12))

def cnt_field_regeneration(X, coupling_levels=np.linspace(0,1,25)):
    """Simulate regeneration from chaos by progressively restoring coupling."""
    # full collapse baseline: shuffle all signals
    X_collapsed = np.random.permutation(X.flatten()).reshape(X.shape)
    base_entropy = cnt_entropy(X)
    collapsed_entropy = cnt_entropy(X_collapsed)
    
    entropies = []
    for c in tqdm(coupling_levels, desc="Reconstructing"):
        blended = (1-c)*X_collapsed + c*X
        entropies.append(cnt_entropy(blended))
    entropies = np.array(entropies)
    
    plt.figure(figsize=(7,4))
    plt.plot(coupling_levels, entropies, 'o-', label='Reconstruction Entropy Path')
    plt.axhline(base_entropy, color='red', ls='--', label=f"Target Entropy (S₀={base_entropy:.3f})")
    plt.axhline(collapsed_entropy, color='gray', ls='--', label=f"Collapsed Entropy (S_c={collapsed_entropy:.3f})")
    plt.xlabel("Coupling Restoration Level")
    plt.ylabel("Entropy")
    plt.title("CNT Reconstruction Simulation — Field Rebirth Test")
    plt.legend()
    plt.show()
    
    recovery_point = coupling_levels[np.argmin(np.abs(entropies - base_entropy))]
    recovery_fraction = (collapsed_entropy - entropies[-1]) / (collapsed_entropy - base_entropy)
    
    return {
        "base_entropy": base_entropy,
        "collapsed_entropy": collapsed_entropy,
        "final_entropy": entropies[-1],
        "recovery_point": recovery_point,
        "recovery_fraction": recovery_fraction
    }

# === Run on your previous field ===
np.random.seed(42)
T, N = 1000, 8
signals = np.array([
    np.sin(np.linspace(0, 10*np.pi, T) + np.random.rand()*5) + np.random.randn(T)*0.1
    for _ in range(N)
]).T

reconstruction = cnt_field_regeneration(signals)
print("=== CNT Reconstruction Results ===")
for k, v in reconstruction.items():
    print(f"{k:20s}: {v:.3f}")


In [ ]:
# === CNT Phase Diagram Generator ===
import numpy as np, matplotlib.pyplot as plt, pandas as pd

# --- Example dataset: your current field ---
data = pd.DataFrame([
    {"Field": "Synthetic Cognitive Field",
     "beta": 1.644,
     "R_n": 0.792,
     "F_r": 1.000}
])

# --- (Optional) Add more fields manually here ---
# data.loc[len(data)] = {"Field": "EEG Alpha Field", "beta": 2.1, "R_n": 0.84, "F_r": 0.72}
# data.loc[len(data)] = {"Field": "Genomic Drift Field", "beta": 0.9, "R_n": 0.68, "F_r": 0.55}

# --- Phase Diagram Plot ---
plt.figure(figsize=(7,6))
sc = plt.scatter(data["beta"], data["R_n"], c=data["F_r"], cmap="viridis", s=200, edgecolor="k")

for i, row in data.iterrows():
    plt.text(row["beta"]+0.02, row["R_n"]+0.01, row["Field"], fontsize=9)

plt.colorbar(sc, label="Recovery Fraction 𝔽ᵣ")
plt.xlabel("Field Sensitivity (β)")
plt.ylabel("Resilience Index (Rₙ)")
plt.title("CNT Entropy–Resonance Phase Diagram")
plt.grid(alpha=0.3)
plt.show()

# --- Quadrant Classification ---
def classify_field(beta, Rn, Fr):
    if Fr > 0.8 and Rn > 0.7:
        return "Resonant–Regenerative Zone (Stable Intelligence)"
    elif Fr > 0.5 and Rn > 0.5:
        return "Metastable Zone (Adaptive Awareness)"
    else:
        return "Chaotic Drift Zone (Entropy-Dominant)"

for i, row in data.iterrows():
    print(f"{row['Field']}: {classify_field(row['beta'], row['R_n'], row['F_r'])}")


In [ ]:
# === CNT Empirical Integration Framework ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit
from tqdm import tqdm

def cnt_entropy(X):
    X = StandardScaler().fit_transform(X)
    corr = np.corrcoef(X.T)
    eigvals, _ = np.linalg.eig(corr)
    eigvals = np.abs(eigvals) / np.sum(np.abs(eigvals))
    return -np.sum(eigvals * np.log(eigvals + 1e-12))

def cnt_phase_metrics(X):
    """Compute key CNT metrics (S0, gamma_c, beta, Rn, Fr) for a real dataset."""
    # Baseline
    S0 = cnt_entropy(X)
    
    # Collapse simulation
    steps = np.linspace(0, 1, 25)
    entropies = []
    for s in tqdm(steps, desc="Simulating collapse"):
        noise = np.random.randn(*X.shape) * s
        entropies.append(cnt_entropy(X + noise))
    entropies = np.array(entropies)
    
    # Collapse threshold
    gamma_c = steps[np.argmax(entropies > S0 + 0.25)]
    Rn = 1 - gamma_c
    
    # Fit β parameter (sensitivity)
    def S_theoretical(gamma, k_r, beta):
        return S0 + k_r * np.exp(beta * (gamma - gamma_c))
    popt, _ = curve_fit(S_theoretical, steps, entropies, p0=[0.25, 1.0])
    _, beta = popt

    # Reconstruction
    X_shuf = np.random.permutation(X.flatten()).reshape(X.shape)
    ent_collapsed = cnt_entropy(X_shuf)
    blend = (1 - 1.0)*X_shuf + 1.0*X
    S_final = cnt_entropy(blend)
    Fr = (ent_collapsed - S_final) / (ent_collapsed - S0)
    
    return {"S0": S0, "beta": beta, "Rn": Rn, "Fr": Fr}

# --- Load your real dataset here ---
# For example: EEG = pd.read_csv("eeg_data.csv").to_numpy()
#              Genome = pd.read_csv("gene_matrix.csv").to_numpy()
# Use a small, numeric matrix for first tests.

# Example (synthetic stand-in until you load real data):
real_data = np.random.randn(1000, 10) * 0.3 + np.sin(np.linspace(0, 20*np.pi, 1000))[:,None]
metrics = cnt_phase_metrics(real_data)
print(metrics)

# --- Add to your phase diagram ---
data.loc[len(data)] = {
    "Field": "Empirical Field (Test)",
    "beta": metrics["beta"],
    "R_n": metrics["Rn"],
    "F_r": metrics["Fr"]
}

plt.figure(figsize=(7,6))
sc = plt.scatter(data["beta"], data["R_n"], c=data["F_r"], cmap="viridis", s=200, edgecolor="k")
for i, row in data.iterrows():
    plt.text(row["beta"]+0.02, row["R_n"]+0.01, row["Field"], fontsize=9)
plt.colorbar(sc, label="Recovery Fraction 𝔽ᵣ")
plt.xlabel("Field Sensitivity (β)")
plt.ylabel("Resilience Index (Rₙ)")
plt.title("CNT Entropy–Resonance Phase Diagram (Synthetic + Real Fields)")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# === CNT Multi-Dataset Expansion Framework ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit
from tqdm import tqdm

# ---- Core Functions ----
def cnt_entropy(X):
    X = StandardScaler().fit_transform(X)
    corr = np.corrcoef(X.T)
    eigvals, _ = np.linalg.eig(corr)
    eigvals = np.abs(eigvals) / np.sum(np.abs(eigvals))
    return -np.sum(eigvals * np.log(eigvals + 1e-12))

def cnt_phase_metrics(X):
    """Return CNT metrics for any field matrix."""
    S0 = cnt_entropy(X)
    steps = np.linspace(0, 1, 25)
    entropies = []
    for s in steps:
        noise = np.random.randn(*X.shape) * s
        entropies.append(cnt_entropy(X + noise))
    entropies = np.array(entropies)
    gamma_c = steps[np.argmax(entropies > S0 + 0.25)]
    Rn = 1 - gamma_c

    def S_theoretical(gamma, k_r, beta):
        return S0 + k_r * np.exp(beta * (gamma - gamma_c))
    popt, _ = curve_fit(S_theoretical, steps, entropies, p0=[0.25, 1.0])
    _, beta = popt

    X_shuf = np.random.permutation(X.flatten()).reshape(X.shape)
    S_collapsed, S_final = cnt_entropy(X_shuf), cnt_entropy(X)
    Fr = (S_collapsed - S_final) / (S_collapsed - S0)
    return dict(S0=S0, beta=beta, Rn=Rn, Fr=Fr)

# ---- Example Datasets (replace with real ones) ----
datasets = {
    "Synthetic Cognitive Field": np.sin(np.linspace(0,10*np.pi,1000))[:,None] + np.random.randn(1000,8)*0.1,
    "EEG Alpha Field (mock)": np.random.randn(1500,16)*0.5 + np.sin(np.linspace(0,40*np.pi,1500))[:,None],
    "Genomic Drift Field (mock)": np.random.randn(800,12)*0.3 + np.linspace(-1,1,800)[:,None],
    "Environmental Oscillation Field (mock)": np.random.randn(1200,10)*0.2 + np.cos(np.linspace(0,15*np.pi,1200))[:,None]
}

# ---- Compute Metrics ----
records = []
for name, X in datasets.items():
    print(f"\nProcessing: {name}")
    metrics = cnt_phase_metrics(X)
    metrics["Field"] = name
    records.append(metrics)

df = pd.DataFrame(records)

# ---- Plot Phase Diagram ----
plt.figure(figsize=(8,6))
sc = plt.scatter(df["beta"], df["Rn"], c=df["Fr"], cmap="plasma", s=200, edgecolor="k")
for _, row in df.iterrows():
    plt.text(row["beta"]+0.02, row["Rn"]+0.01, row["Field"], fontsize=9)
plt.colorbar(sc, label="Recovery Fraction 𝔽ᵣ")
plt.xlabel("Field Sensitivity (β)")
plt.ylabel("Resilience Index (Rₙ)")
plt.title("CNT Multi-Domain Entropy–Resonance Phase Atlas")
plt.grid(alpha=0.3)
plt.show()

print(df[["Field","beta","Rn","Fr"]])


In [ ]:
# === CNT Phase-Surface Modeling ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# --- Assume df from your Atlas phase step already exists ---
# Columns: ["Field","beta","Rn","Fr"]

# Build interpolation grid
beta_grid  = np.linspace(df["beta"].min()-0.05, df["beta"].max()+0.05, 80)
Rn_grid    = np.linspace(df["Rn"].min()-0.05,   df["Rn"].max()+0.05,   80)
B, R       = np.meshgrid(beta_grid, Rn_grid)
F_interp   = griddata(
    (df["beta"], df["Rn"]), df["Fr"],
    (B, R), method="cubic"
)

# --- 3-D Surface Plot ---
fig = plt.figure(figsize=(8,6))
ax  = fig.add_subplot(111, projection="3d")
surf = ax.plot_surface(B, R, F_interp, cmap="plasma", edgecolor="none", alpha=0.9)
ax.set_xlabel("Field Sensitivity β")
ax.set_ylabel("Resilience Index Rₙ")
ax.set_zlabel("Recovery Fraction 𝔽ᵣ")
ax.set_title("CNT 3-D Phase-Surface — Stability Landscape")
fig.colorbar(surf, ax=ax, shrink=0.5, label="𝔽ᵣ Intensity")
plt.show()


In [ ]:
# === CNT Curvature and Stability Basin Analysis ===
import numpy as np, matplotlib.pyplot as plt
from numpy import gradient

# Assume df (or df_ext if you added extra points) already exists
# Columns: beta, Rn, Fr

# ---- Build smooth grid ----
beta_grid = np.linspace(df["beta"].min()-0.05, df["beta"].max()+0.05, 100)
Rn_grid   = np.linspace(df["Rn"].min()-0.05,   df["Rn"].max()+0.05,   100)
B, R = np.meshgrid(beta_grid, Rn_grid)

# Interpolate F values over the grid
from scipy.interpolate import griddata
F = griddata((df["beta"], df["Rn"]), df["Fr"], (B, R), method="cubic")

# ---- Compute gradients and curvature ----
dF_dB, dF_dR = gradient(F, beta_grid, Rn_grid)
d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, Rn_grid)[0], gradient(dF_dR, beta_grid, Rn_grid)[1]
curvature = d2F_dB2 + d2F_dR2     # Laplacian ≈ total curvature

# ---- Visualization: Curvature Map ----
plt.figure(figsize=(8,6))
curv_plot = plt.contourf(B, R, curvature, levels=40, cmap="coolwarm")
plt.colorbar(curv_plot, label="Curvature (∇²𝔽ᵣ)")
plt.contour(B, R, F, colors='k', linewidths=0.5, alpha=0.5)
plt.xlabel("Field Sensitivity β")
plt.ylabel("Resilience Index Rₙ")
plt.title("CNT Stability Basin Curvature Map")
plt.grid(alpha=0.2)
plt.show()

# ---- Identify Basin and Ridge Regions ----
mean_curv = np.nanmean(curvature)
stable_mask = curvature > mean_curv
unstable_mask = curvature < mean_curv

stable_pct = np.sum(stable_mask) / np.size(curvature) * 100
unstable_pct = np.sum(unstable_mask) / np.size(curvature) * 100

print("=== CNT Curvature Summary ===")
print(f"Mean curvature: {mean_curv:.5f}")
print(f"Stable-basin area  (∇²𝔽ᵣ > mean): {stable_pct:.1f}% of surface")
print(f"Unstable-ridge area (∇²𝔽ᵣ < mean): {unstable_pct:.1f}% of surface")


In [ ]:
# === CNT Phase–Curvature Meta-Analysis ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.interpolate import griddata
from numpy import gradient

# ------------------------------------------------------------
# 1. TEMPORAL CURVATURE DRIFT  (Evolution over epochs)
# ------------------------------------------------------------
def cnt_curvature_evolution(data_epochs):
    """
    data_epochs: dict{name: pd.DataFrame with columns beta,Rn,Fr for each time slice}
    returns: dict{name: curvature_map}
    """
    results = {}
    for label, df_epoch in data_epochs.items():
        beta_grid = np.linspace(df_epoch["beta"].min()-0.05, df_epoch["beta"].max()+0.05, 80)
        Rn_grid   = np.linspace(df_epoch["Rn"].min()-0.05,   df_epoch["Rn"].max()+0.05,   80)
        B, R = np.meshgrid(beta_grid, Rn_grid)
        F = griddata((df_epoch["beta"], df_epoch["Rn"]), df_epoch["Fr"], (B, R), method="cubic")
        dF_dB, dF_dR = gradient(F, beta_grid, Rn_grid)
        d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, Rn_grid)[0], gradient(dF_dR, beta_grid, Rn_grid)[1]
        curvature = d2F_dB2 + d2F_dR2
        results[label] = curvature

        plt.figure(figsize=(7,5))
        curv_plot = plt.contourf(B, R, curvature, levels=40, cmap="coolwarm")
        plt.colorbar(curv_plot, label="Curvature ∇²𝔽ᵣ")
        plt.title(f"CNT Curvature Evolution — Epoch: {label}")
        plt.xlabel("β (Sensitivity)")
        plt.ylabel("Rₙ (Resilience)")
        plt.grid(alpha=0.2)
        plt.show()
    return results

# Example: simulate 3 epochs (e.g., system before / during / after stress)
# data_epoch1 = df.copy(); data_epoch2 = df.copy(); data_epoch3 = df.copy()
# curvature_maps = cnt_curvature_evolution({"Pre-Stimulus": data_epoch1, "Active": data_epoch2, "Recovery": data_epoch3})


# ------------------------------------------------------------
# 2. CROSS-DOMAIN COMPARISON (multi-field curvature overlay)
# ------------------------------------------------------------
def cnt_cross_domain_curvature(domains):
    """
    domains: dict{name: pd.DataFrame with beta,Rn,Fr for each field}
    overlays curvature maps for visual comparison
    """
    plt.figure(figsize=(8,6))
    for label, df_domain in domains.items():
        beta_grid = np.linspace(df_domain["beta"].min()-0.05, df_domain["beta"].max()+0.05, 80)
        Rn_grid   = np.linspace(df_domain["Rn"].min()-0.05,   df_domain["Rn"].max()+0.05,   80)
        B, R = np.meshgrid(beta_grid, Rn_grid)
        F = griddata((df_domain["beta"], df_domain["Rn"]), df_domain["Fr"], (B, R), method="cubic")
        dF_dB, dF_dR = gradient(F, beta_grid, Rn_grid)
        d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, Rn_grid)[0], gradient(dF_dR, beta_grid, Rn_grid)[1]
        curvature = d2F_dB2 + d2F_dR2
        plt.contour(B, R, curvature, levels=[0], linewidths=1.5, label=label)
    plt.xlabel("β (Sensitivity)")
    plt.ylabel("Rₙ (Resilience)")
    plt.title("CNT Cross-Domain Curvature Overlay (∇²𝔽ᵣ ≈ 0 contours)")
    plt.legend(domains.keys())
    plt.grid(alpha=0.3)
    plt.show()

# Example: cnt_cross_domain_curvature({"Biological": df_bio, "Synthetic": df_syn, "Environmental": df_env})


# ------------------------------------------------------------
# 3. GLOBAL STABILITY POTENTIAL  (integrated resilience energy)
# ------------------------------------------------------------
def cnt_global_stability_potential(df):
    """
    Integrate −∇²𝔽ᵣ over the surface to estimate total stability energy.
    """
    beta_grid = np.linspace(df["beta"].min()-0.05, df["beta"].max()+0.05, 120)
    Rn_grid   = np.linspace(df["Rn"].min()-0.05,   df["Rn"].max()+0.05,   120)
    B, R = np.meshgrid(beta_grid, Rn_grid)
    F = griddata((df["beta"], df["Rn"]), df["Fr"], (B, R), method="cubic")
    dF_dB, dF_dR = gradient(F, beta_grid, Rn_grid)
    d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, Rn_grid)[0], gradient(dF_dR, beta_grid, Rn_grid)[1]
    curvature = d2F_dB2 + d2F_dR2
    stability_potential = -np.nanmean(curvature)
    area = np.product([beta_grid.ptp(), Rn_grid.ptp()])
    total_resilience_energy = stability_potential * area

    print("=== CNT Global Stability Potential ===")
    print(f"Mean curvature (−∇²𝔽ᵣ): {stability_potential:.6e}")
    print(f"Phase-plane area: {area:.6f}")
    print(f"Total resilience energy: {total_resilience_energy:.6e}")
    return stability_potential, total_resilience_energy

# Example:
# potential, energy = cnt_global_stability_potential(df_ext)


In [ ]:
# === DEMO: Temporal Curvature Evolution ===
data_epoch1 = df.copy()
data_epoch2 = df.copy(); data_epoch2["Fr"] *= 0.97  # simulate mild degradation
data_epoch3 = df.copy(); data_epoch3["Fr"] *= 1.02  # simulate recovery

curvature_maps = cnt_curvature_evolution({
    "Pre-Stimulus": data_epoch1,
    "Active": data_epoch2,
    "Recovery": data_epoch3
})


In [ ]:
# === DEMO: Cross-Domain Curvature Overlay ===
cnt_cross_domain_curvature({
    "Synthetic": df.query("Field == 'Synthetic Cognitive Field'"),
    "EEG": df.query("Field == 'EEG Alpha Field (mock)'"),
    "Genomic": df.query("Field == 'Genomic Drift Field (mock)'"),
    "Environmental": df.query("Field == 'Environmental Oscillation Field (mock)'")
})


In [ ]:
# === DEMO: Global Stability Potential ===
potential, energy = cnt_global_stability_potential(df)
print(f"Mean curvature: {potential:.3e}")
print(f"Integrated stability energy: {energy:.3e}")


In [ ]:
# === CNT Cross-Domain Curvature + Global Stability Potential (One Cell) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.interpolate import griddata
from numpy import gradient

# --- Create richer multi-sample data for each domain ---
domains = {
    "Synthetic": pd.DataFrame({
        "beta": 1.68 + 0.02*np.random.randn(6),
        "Rn":   0.79 + 0.01*np.random.randn(6),
        "Fr":   1.00 + 0.00*np.random.randn(6)
    }),
    "EEG": pd.DataFrame({
        "beta": 2.02 + 0.03*np.random.randn(6),
        "Rn":   0.67 + 0.02*np.random.randn(6),
        "Fr":   0.93 + 0.02*np.random.randn(6)
    }),
    "Genomic": pd.DataFrame({
        "beta": 1.72 + 0.02*np.random.randn(6),
        "Rn":   0.75 + 0.02*np.random.randn(6),
        "Fr":   0.96 + 0.01*np.random.randn(6)
    }),
    "Environmental": pd.DataFrame({
        "beta": 1.79 + 0.02*np.random.randn(6),
        "Rn":   0.79 + 0.01*np.random.randn(6),
        "Fr":   0.98 + 0.01*np.random.randn(6)
    })
}

# --- Combine all into one DataFrame for potential calculation ---
df = pd.concat([pd.concat([v.assign(Field=k) for k,v in domains.items()])], ignore_index=True)

# --- Function: cross-domain curvature overlay ---
def cnt_cross_domain_curvature(domains):
    plt.figure(figsize=(8,6))
    for label, df_domain in domains.items():
        beta_grid = np.linspace(df_domain["beta"].min()-0.05, df_domain["beta"].max()+0.05, 60)
        Rn_grid   = np.linspace(df_domain["Rn"].min()-0.05,   df_domain["Rn"].max()+0.05,   60)
        B, R = np.meshgrid(beta_grid, Rn_grid)
        F = griddata((df_domain["beta"], df_domain["Rn"]), df_domain["Fr"],
                     (B, R), method="cubic")
        dF_dB, dF_dR = gradient(F, beta_grid, Rn_grid)
        d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, Rn_grid)[0], gradient(dF_dR, beta_grid, Rn_grid)[1]
        curvature = d2F_dB2 + d2F_dR2
        plt.contour(B, R, curvature, levels=[0], linewidths=1.5, label=label)
    plt.xlabel("β (Sensitivity)")
    plt.ylabel("Rₙ (Resilience)")
    plt.title("CNT Cross-Domain Curvature Overlay (∇²𝔽ᵣ ≈ 0 Contours)")
    plt.legend(domains.keys())
    plt.grid(alpha=0.3)
    plt.show()

# --- Function: global stability potential ---
def cnt_global_stability_potential(df):
    beta_grid = np.linspace(df["beta"].min()-0.05, df["beta"].max()+0.05, 120)
    Rn_grid   = np.linspace(df["Rn"].min()-0.05,   df["Rn"].max()+0.05,   120)
    B, R = np.meshgrid(beta_grid, Rn_grid)
    F = griddata((df["beta"], df["Rn"]), df["Fr"], (B, R), method="cubic")
    dF_dB, dF_dR = gradient(F, beta_grid, Rn_grid)
    d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, Rn_grid)[0], gradient(dF_dR, beta_grid, Rn_grid)[1]
    curvature = d2F_dB2 + d2F_dR2
    stability_potential = -np.nanmean(curvature)
    area = np.prod([beta_grid.ptp(), Rn_grid.ptp()])
    total_resilience_energy = stability_potential * area
    print("=== CNT Global Stability Potential ===")
    print(f"Mean curvature (−∇²𝔽ᵣ): {stability_potential:.6e}")
    print(f"Phase-plane area: {area:.6f}")
    print(f"Total resilience energy: {total_resilience_energy:.6e}")
    return stability_potential, total_resilience_energy

# --- Execute both analyses ---
cnt_cross_domain_curvature(domains)
potential, energy = cnt_global_stability_potential(df)


In [ ]:
# === CNT Cross-Domain Curvature + Global Stability Potential (NumPy 2.0 safe, robust) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.interpolate import griddata
from numpy import gradient

# ----- helper: robust interpolation with cubic→linear fallback and NaN mask -----
def safe_interp(beta, rn, fr, B, R, method="cubic"):
    F = griddata((beta, rn), fr, (B, R), method=method)
    # fallback if cubic fails or returns too many NaNs
    if F is None or np.all(np.isnan(F)):
        F = griddata((beta, rn), fr, (B, R), method="linear")
    return F

# ----- synthesize richer multi-sample domains (replace with your real data anytime) -----
rng = np.random.default_rng(7)
domains = {
    "Synthetic": pd.DataFrame({
        "beta": 1.68 + 0.02*rng.standard_normal(8),
        "Rn":   0.79 + 0.010*rng.standard_normal(8),
        "Fr":   1.00 + 0.002*rng.standard_normal(8)
    }),
    "EEG": pd.DataFrame({
        "beta": 2.02 + 0.03*rng.standard_normal(8),
        "Rn":   0.67 + 0.020*rng.standard_normal(8),
        "Fr":   0.93 + 0.020*rng.standard_normal(8)
    }),
    "Genomic": pd.DataFrame({
        "beta": 1.72 + 0.02*rng.standard_normal(8),
        "Rn":   0.75 + 0.020*rng.standard_normal(8),
        "Fr":   0.96 + 0.010*rng.standard_normal(8)
    }),
    "Environmental": pd.DataFrame({
        "beta": 1.79 + 0.02*rng.standard_normal(8),
        "Rn":   0.79 + 0.010*rng.standard_normal(8),
        "Fr":   0.98 + 0.010*rng.standard_normal(8)
    })
}

# ----- combine for global metrics -----
df = pd.concat([v.assign(Field=k) for k, v in domains.items()], ignore_index=True)

# ----- cross-domain zero-curvature overlay -----
plt.figure(figsize=(8,6))
for label, d in domains.items():
    beta_grid = np.linspace(d["beta"].min()-0.06, d["beta"].max()+0.06, 80)
    rn_grid   = np.linspace(d["Rn"].min()-0.06,   d["Rn"].max()+0.06,   80)
    B, R = np.meshgrid(beta_grid, rn_grid)
    F = safe_interp(d["beta"].to_numpy(), d["Rn"].to_numpy(), d["Fr"].to_numpy(), B, R, method="cubic")

    # mask NaNs to keep gradients stable
    Fm = np.where(np.isnan(F), np.nanmean(F), F)
    dF_dB, dF_dR = gradient(Fm, beta_grid, rn_grid)
    d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, rn_grid)[0], gradient(dF_dR, beta_grid, rn_grid)[1]
    curv = d2F_dB2 + d2F_dR2

    # draw multiple near-zero contours to improve visibility
    levels = [-1e-10, -5e-11, 0, 5e-11, 1e-10]
    cs = plt.contour(B, R, curv, levels=levels, linewidths=1.2, alpha=0.9)
    # label the middle (≈0) line with domain name
    if cs.collections:
        cs0 = cs.collections[len(levels)//2]
        if cs0.get_paths():
            path = cs0.get_paths()[0]
            v = path.vertices[len(path.vertices)//2]
            plt.text(v[0], v[1], label, fontsize=9, weight="bold")

plt.xlabel("β (Sensitivity)")
plt.ylabel("Rₙ (Resilience)")
plt.title("CNT Cross-Domain Curvature Overlay (∇²𝔽ᵣ ≈ 0 Contours)")
plt.grid(alpha=0.3)
plt.show()

# ----- global stability potential (NumPy 2.0 safe) -----
beta_grid = np.linspace(df["beta"].min()-0.08, df["beta"].max()+0.08, 150)
rn_grid   = np.linspace(df["Rn"].min()-0.08,   df["Rn"].max()+0.08,   150)
B, R = np.meshgrid(beta_grid, rn_grid)
F = safe_interp(df["beta"].to_numpy(), df["Rn"].to_numpy(), df["Fr"].to_numpy(), B, R, method="cubic")
Fm = np.where(np.isnan(F), np.nanmean(F), F)
dF_dB, dF_dR = gradient(Fm, beta_grid, rn_grid)
d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, rn_grid)[0], gradient(dF_dR, beta_grid, rn_grid)[1]
curv = d2F_dB2 + d2F_dR2

stability_potential = -np.nanmean(curv)       # −∇²F̄ → restorative tendency
area = np.ptp(beta_grid) * np.ptp(rn_grid)    # NumPy 2.0: use np.ptp(arr)
total_resilience_energy = stability_potential * area

print("=== CNT Global Stability Potential ===")
print(f"Mean curvature (−∇²𝔽ᵣ): {stability_potential:.6e}")
print(f"Phase-plane area:         {area:.6f}")
print(f"Total resilience energy:  {total_resilience_energy:.6e}")


In [ ]:
# === CNT Overlay (centroid labels) + Global Stability Potential — robust one-cell patch ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.interpolate import griddata
from numpy import gradient

# ---- safety: robust interpolation with cubic→linear fallback and NaN fill ----
def safe_interp(beta, rn, fr, B, R, method="cubic"):
    F = griddata((beta, rn), fr, (B, R), method=method)
    if F is None or np.all(np.isnan(F)):
        F = griddata((beta, rn), fr, (B, R), method="linear")
    if F is None:
        # last resort: nearest
        F = griddata((beta, rn), fr, (B, R), method="nearest")
    # replace any residual NaNs with global mean to stabilize gradients
    if np.isnan(F).any():
        F = np.where(np.isnan(F), np.nanmean(F), F)
    return F

# ---- if you already have `domains` from earlier, we’ll reuse it; otherwise synthesize quick samples
try:
    domains
except NameError:
    rng = np.random.default_rng(7)
    domains = {
        "Synthetic": pd.DataFrame({
            "beta": 1.68 + 0.02*rng.standard_normal(8),
            "Rn":   0.79 + 0.010*rng.standard_normal(8),
            "Fr":   1.00 + 0.002*rng.standard_normal(8)
        }),
        "EEG": pd.DataFrame({
            "beta": 2.02 + 0.03*rng.standard_normal(8),
            "Rn":   0.67 + 0.020*rng.standard_normal(8),
            "Fr":   0.93 + 0.020*rng.standard_normal(8)
        }),
        "Genomic": pd.DataFrame({
            "beta": 1.72 + 0.02*rng.standard_normal(8),
            "Rn":   0.75 + 0.020*rng.standard_normal(8),
            "Fr":   0.96 + 0.010*rng.standard_normal(8)
        }),
        "Environmental": pd.DataFrame({
            "beta": 1.79 + 0.02*rng.standard_normal(8),
            "Rn":   0.79 + 0.010*rng.standard_normal(8),
            "Fr":   0.98 + 0.010*rng.standard_normal(8)
        })
    }

# ---- overlay with centroid labels (no reliance on QuadContourSet internals)
plt.figure(figsize=(8,6))

# choose consistent colors per domain
palette = {
    "Synthetic":        "#1f77b4",
    "EEG":              "#d62728",
    "Genomic":          "#2ca02c",
    "Environmental":    "#9467bd"
}

# determine global bounds to avoid edge artifacts
all_beta = np.concatenate([d["beta"].to_numpy() for d in domains.values()])
all_rn   = np.concatenate([d["Rn"].to_numpy()   for d in domains.values()])
bx = (all_beta.min()-0.06, all_beta.max()+0.06)
ry = (all_rn.min()-0.06,   all_rn.max()+0.06)

for label, d in domains.items():
    beta_grid = np.linspace(max(bx[0], d["beta"].min()-0.05),
                            min(bx[1], d["beta"].max()+0.05), 100)
    rn_grid   = np.linspace(max(ry[0], d["Rn"].min()-0.05),
                            min(ry[1], d["Rn"].max()+0.05),   100)
    B, R = np.meshgrid(beta_grid, rn_grid)
    F = safe_interp(d["beta"].to_numpy(), d["Rn"].to_numpy(), d["Fr"].to_numpy(), B, R, method="cubic")

    dF_dB, dF_dR = gradient(F, beta_grid, rn_grid)
    d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, rn_grid)[0], gradient(dF_dR, beta_grid, rn_grid)[1]
    curv = d2F_dB2 + d2F_dR2

    # draw a near-zero band for visibility
    levels = [-5e-11, 0, 5e-11]
    plt.contour(B, R, curv, levels=levels, colors=palette.get(label, "k"),
                linewidths=(0.8, 1.6, 0.8), alpha=0.95)

    # label at centroid of the domain samples
    c_beta, c_rn = float(d["beta"].mean()), float(d["Rn"].mean())
    plt.annotate(label, xy=(c_beta, c_rn), xytext=(4,4), textcoords="offset points",
                 fontsize=9, weight="bold", color=palette.get(label, "k"))

plt.xlabel("β (Sensitivity)")
plt.ylabel("Rₙ (Resilience)")
plt.title("CNT Cross-Domain Curvature Overlay (∇²𝔽ᵣ ≈ 0 Band)")
plt.grid(alpha=0.3)
plt.xlim(bx); plt.ylim(ry)
plt.show()

# ---- Global stability potential (NumPy 2.0 safe) ----
df_all = pd.concat([v.assign(Field=k) for k, v in domains.items()], ignore_index=True)
gb = np.linspace(bx[0], bx[1], 160)
gr = np.linspace(ry[0], ry[1], 160)
B, R = np.meshgrid(gb, gr)
F = safe_interp(df_all["beta"].to_numpy(), df_all["Rn"].to_numpy(), df_all["Fr"].to_numpy(), B, R, method="cubic")
dF_dB, dF_dR = gradient(F, gb, gr)
d2F_dB2, d2F_dR2 = gradient(dF_dB, gb, gr)[0], gradient(dF_dR, gb, gr)[1]
curv = d2F_dB2 + d2F_dR2

stability_potential = -np.nanmean(curv)          # restorative tendency
area = np.ptp(gb) * np.ptp(gr)                    # NumPy 2.0: use np.ptp
total_resilience_energy = stability_potential * area

print("=== CNT Global Stability Potential ===")
print(f"Mean curvature (−∇²𝔽ᵣ): {stability_potential:.6e}")
print(f"Phase-plane area:         {area:.6f}")
print(f"Total resilience energy:  {total_resilience_energy:.6e}")


In [ ]:
# === Clean Overlay + Bootstrap Centroids (smoothing, clearer bands, CIs) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter
from numpy import gradient

def safe_interp(beta, rn, fr, B, R):
    F = griddata((beta, rn), fr, (B, R), method="cubic")
    if F is None or np.all(np.isnan(F)):
        F = griddata((beta, rn), fr, (B, R), method="linear")
    if F is None:  # nearest as last resort
        F = griddata((beta, rn), fr, (B, R), method="nearest")
    F = np.where(np.isnan(F), np.nanmean(F), F)
    return F

def bootstrap_centroid(df, n=500, alpha=0.05, rng=None):
    rng = np.random.default_rng(rng)
    pts = []
    arr = df[["beta","Rn"]].to_numpy()
    for _ in range(n):
        idx = rng.integers(0, len(arr), len(arr))
        pts.append(arr[idx].mean(axis=0))
    pts = np.array(pts)
    lo = np.quantile(pts, alpha/2, axis=0)
    hi = np.quantile(pts, 1-alpha/2, axis=0)
    return arr.mean(axis=0), lo, hi

# use your existing `domains` dict (from last cell)
palette = {
    "Synthetic": "#1f77b4",
    "EEG": "#d62728",
    "Genomic": "#2ca02c",
    "Environmental": "#9467bd"
}

# global bounds from all domains
all_beta = np.concatenate([d["beta"].to_numpy() for d in domains.values()])
all_rn   = np.concatenate([d["Rn"].to_numpy()   for d in domains.values()])
bx = (all_beta.min()-0.06, all_beta.max()+0.06)
ry = (all_rn.min()-0.06,   all_rn.max()+0.06)

plt.figure(figsize=(8,6))
for name, d in domains.items():
    beta_grid = np.linspace(max(bx[0], d["beta"].min()-0.05),
                            min(bx[1], d["beta"].max()+0.05), 140)
    rn_grid   = np.linspace(max(ry[0], d["Rn"].min()-0.05),
                            min(ry[1], d["Rn"].max()+0.05),   140)
    B, R = np.meshgrid(beta_grid, rn_grid)
    F = safe_interp(d["beta"].to_numpy(), d["Rn"].to_numpy(), d["Fr"].to_numpy(), B, R)

    # smooth slightly before curvature to remove striping (keep sigma small)
    F_smooth = gaussian_filter(F, sigma=1.2)
    dF_dB, dF_dR = gradient(F_smooth, beta_grid, rn_grid)
    d2F_dB2, d2F_dR2 = gradient(dF_dB, beta_grid, rn_grid)[0], gradient(dF_dR, beta_grid, rn_grid)[1]
    curv = d2F_dB2 + d2F_dR2

    # draw a clearer near-zero band
    levels = [-1e-11, 0, 1e-11]
    plt.contour(B, R, curv, levels=levels, colors=palette.get(name, "k"),
                linewidths=(0.8, 2.0, 0.8), alpha=0.95)

    # bootstrap centroid with 95% CI
    cen, lo, hi = bootstrap_centroid(d, n=600, rng=7)
    plt.errorbar(cen[0], cen[1],
                 xerr=[[cen[0]-lo[0]], [hi[0]-cen[0]]],
                 yerr=[[cen[1]-lo[1]], [hi[1]-cen[1]]],
                 fmt='o', color=palette.get(name, "k"), capsize=3, label=f"{name} centroid")

plt.xlabel("β (Sensitivity)")
plt.ylabel("Rₙ (Resilience)")
plt.title("CNT Curvature Overlay — smoothed, with bootstrap centroids (95% CI)")
plt.grid(alpha=0.3); plt.xlim(bx); plt.ylim(ry)
plt.legend(fontsize=8, loc="best")
plt.show()


In [ ]:
# === CNT Phase Report: table + potential + exports (one cell) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter
from numpy import gradient

# expects `domains` dict from earlier; keys -> DataFrames with columns [beta, Rn, Fr]

def bootstrap_centroid(df, n=1000, alpha=0.05, rng=7):
    rng = np.random.default_rng(rng)
    arr = df[["beta","Rn"]].to_numpy()
    boots = np.stack([arr[rng.integers(0, len(arr), len(arr))].mean(axis=0) for _ in range(n)])
    cen = arr.mean(axis=0)
    lo  = np.quantile(boots, alpha/2, axis=0)
    hi  = np.quantile(boots, 1-alpha/2, axis=0)
    return cen, lo, hi

def safe_interp(beta, rn, fr, B, R):
    F = griddata((beta, rn), fr, (B, R), method="cubic")
    if F is None or np.all(np.isnan(F)):
        F = griddata((beta, rn), fr, (B, R), method="linear")
    if F is None:
        F = griddata((beta, rn), fr, (B, R), method="nearest")
    return np.where(np.isnan(F), np.nanmean(F), F)

# ---------- 1) Metrics table with CIs ----------
rows = []
for name, d in domains.items():
    cen, lo, hi = bootstrap_centroid(d)
    rows.append({
        "Field": name,
        "beta_mean": cen[0], "beta_lo": lo[0], "beta_hi": hi[0],
        "Rn_mean": cen[1],   "Rn_lo": lo[1],   "Rn_hi": hi[1],
        "Fr_mean": float(d["Fr"].mean())
    })
report = pd.DataFrame(rows).sort_values("beta_mean").reset_index(drop=True)
print("=== CNT Phase Report (centroid ± 95% CI) ===")
print(report.to_string(index=False))

# ---------- 2) Global stability potential ----------
df_all = pd.concat([v.assign(Field=k) for k, v in domains.items()], ignore_index=True)
bx = (df_all["beta"].min()-0.06, df_all["beta"].max()+0.06)
ry = (df_all["Rn"].min()-0.06,   df_all["Rn"].max()+0.06)
gb = np.linspace(bx[0], bx[1], 160)
gr = np.linspace(ry[0], ry[1], 160)
B, R = np.meshgrid(gb, gr)
F = safe_interp(df_all["beta"].to_numpy(), df_all["Rn"].to_numpy(), df_all["Fr"].to_numpy(), B, R)
F = gaussian_filter(F, sigma=1.2)  # gentle smoothing for stable curvature
dF_dB, dF_dR = gradient(F, gb, gr)
d2F_dB2, d2F_dR2 = gradient(dF_dB, gb, gr)[0], gradient(dF_dR, gb, gr)[1]
curv = d2F_dB2 + d2F_dR2

stability_potential = -np.nanmean(curv)
area = np.ptp(gb) * np.ptp(gr)  # NumPy 2.0
total_resilience_energy = stability_potential * area
print("\n=== CNT Global Stability Potential ===")
print(f"Mean curvature (−∇²𝔽ᵣ): {stability_potential:.6e}")
print(f"Phase-plane area:         {area:.6f}")
print(f"Total resilience energy:  {total_resilience_energy:.6e}")

# ---------- 3) Save outputs ----------
out_dir = "cnt_phase_exports"
import os; os.makedirs(out_dir, exist_ok=True)

# save table
csv_path = os.path.join(out_dir, "cnt_phase_report.csv")
report.to_csv(csv_path, index=False)

# save figure (replot smoothed overlay quickly)
palette = {"Synthetic": "#1f77b4","EEG": "#d62728","Genomic": "#2ca02c","Environmental": "#9467bd"}
plt.figure(figsize=(8,6))
for name, d in domains.items():
    beta_grid = np.linspace(max(bx[0], d["beta"].min()-0.05), min(bx[1], d["beta"].max()+0.05), 140)
    rn_grid   = np.linspace(max(ry[0], d["Rn"].min()-0.05),   min(ry[1], d["Rn"].max()+0.05),   140)
    B2, R2 = np.meshgrid(beta_grid, rn_grid)
    F2 = safe_interp(d["beta"].to_numpy(), d["Rn"].to_numpy(), d["Fr"].to_numpy(), B2, R2)
    F2 = gaussian_filter(F2, sigma=1.2)
    dB, dR = gradient(F2, beta_grid, rn_grid)
    c = gradient(dB, beta_grid, rn_grid)[0] + gradient(dR, beta_grid, rn_grid)[1]
    plt.contour(B2, R2, c, levels=[-1e-11, 0, 1e-11], colors=palette.get(name,"k"),
                linewidths=(0.8, 2.0, 0.8), alpha=0.95, linestyles="solid")
    cen,_lo,_hi = bootstrap_centroid(d)
    plt.plot(cen[0], cen[1], "o", color=palette.get(name,"k"), label=f"{name} centroid")
plt.xlabel("β (Sensitivity)"); plt.ylabel("Rₙ (Resilience)")
plt.title("CNT Curvature Overlay — smoothed (saved)")
plt.grid(alpha=0.3); plt.xlim(bx); plt.ylim(ry)
plt.legend(fontsize=8, loc="best")
fig_path = os.path.join(out_dir, "cnt_curvature_overlay.png")
plt.savefig(fig_path, dpi=180, bbox_inches="tight"); plt.show()

print(f"\nSaved: {csv_path}\nSaved: {fig_path}")


In [ ]:
# === Real Data → CNT Metrics → Phase Diagram & Report (append alongside existing domains) ===
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter
from numpy import gradient

# ---------- CONFIG: add your real matrices here ----------
# Each entry: {"name": "...", "path": "...", "loader": "csv"|"npy", "has_header": True/False}
real_sources = [
    # EXAMPLES — replace with your files:
    # {"name": "EEG_SessionA", "path": r"C:\Users\caleb\data\eeg_sessionA.csv", "loader": "csv", "has_header": True},
    # {"name": "HiC_chr1_50kb", "path": r"C:\Users\caleb\data\hic_chr1_50kb.npy", "loader": "npy", "has_header": False},
]

# How many columns (channels/features) to keep (optional downselect for huge files)
max_cols = 64   # set None to keep all

# ---------- CNT helpers (re-useable) ----------
from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit

def cnt_entropy(X):
    X = StandardScaler().fit_transform(X)
    C = np.corrcoef(X.T)
    w, _ = np.linalg.eig(C)
    p = np.abs(w) / np.sum(np.abs(w))
    return -np.sum(p * np.log(p + 1e-12))

def cnt_phase_metrics(X):
    S0 = cnt_entropy(X)
    steps = np.linspace(0, 1, 25)
    ent = []
    for s in steps:
        ent.append(cnt_entropy(X + np.random.randn(*X.shape)*s))
    ent = np.array(ent)
    gamma_c = steps[np.argmax(ent > S0 + 0.25)]
    Rn = 1 - gamma_c
    def S_theo(g, k_r, beta):
        return S0 + k_r*np.exp(beta*(g - gamma_c))
    (k_r, beta), _ = curve_fit(S_theo, steps, ent, p0=[0.25, 1.0], maxfev=10000)
    X_shuf = np.random.permutation(X.flatten()).reshape(X.shape)
    S_collapsed, S_final = cnt_entropy(X_shuf), cnt_entropy(X)
    Fr = (S_collapsed - S_final) / (S_collapsed - S0)
    return {"S0": S0, "beta": float(beta), "Rn": float(Rn), "Fr": float(Fr)}

def safe_interp(beta, rn, fr, B, R):
    F = griddata((beta, rn), fr, (B, R), method="cubic")
    if F is None or np.all(np.isnan(F)):
        F = griddata((beta, rn), fr, (B, R), method="linear")
    if F is None:
        F = griddata((beta, rn), fr, (B, R), method="nearest")
    return np.where(np.isnan(F), np.nanmean(F), F)

def bootstrap_centroid(df, n=1000, alpha=0.05, rng=7):
    rng = np.random.default_rng(rng)
    arr = df[["beta","Rn"]].to_numpy()
    boots = np.stack([arr[rng.integers(0, len(arr), len(arr))].mean(axis=0) for _ in range(n)])
    cen = arr.mean(axis=0); lo = np.quantile(boots, alpha/2, axis=0); hi = np.quantile(boots, 1-alpha/2, axis=0)
    return cen, lo, hi

# ---------- 1) Load real datasets and compute metrics ----------
new_domains = {}
for spec in real_sources:
    if spec["loader"] == "csv":
        df_raw = pd.read_csv(spec["path"]) if spec.get("has_header", True) else pd.read_csv(spec["path"], header=None)
        X = df_raw.to_numpy()
    elif spec["loader"] == "npy":
        X = np.load(spec["path"])
    else:
        raise ValueError(f"Unknown loader: {spec['loader']}")

    if max_cols and X.shape[1] > max_cols:
        X = X[:, :max_cols]

    m = cnt_phase_metrics(X)
    # small jittered cloud around centroid so surfaces can be interpolated
    rng = np.random.default_rng(42)
    cloud = pd.DataFrame({
        "beta": m["beta"] + 0.015*rng.standard_normal(10),
        "Rn":   m["Rn"]   + 0.015*rng.standard_normal(10),
        "Fr":   np.clip(m["Fr"] + 0.01*rng.standard_normal(10), 0, 1.1)
    })
    new_domains[spec["name"]] = cloud

# Merge with existing `domains` dict if present; otherwise start with new
try:
    domains
    domains = {**domains, **new_domains}
except NameError:
    domains = new_domains

# ---------- 2) Regenerate smoothed curvature overlay with centroids ----------
palette = {}
for i, k in enumerate(domains.keys()):
    # assign deterministic distinct colors
    palette[k] = plt.cm.tab10(i % 10)

all_beta = np.concatenate([d["beta"].to_numpy() for d in domains.values()])
all_rn   = np.concatenate([d["Rn"].to_numpy()   for d in domains.values()])
bx = (all_beta.min()-0.06, all_beta.max()+0.06)
ry = (all_rn.min()-0.06,   all_rn.max()+0.06)

plt.figure(figsize=(8,6))
for name, d in domains.items():
    beta_grid = np.linspace(max(bx[0], d["beta"].min()-0.05), min(bx[1], d["beta"].max()+0.05), 140)
    rn_grid   = np.linspace(max(ry[0], d["Rn"].min()-0.05),   min(ry[1], d["Rn"].max()+0.05),   140)
    B, R = np.meshgrid(beta_grid, rn_grid)
    F = safe_interp(d["beta"].to_numpy(), d["Rn"].to_numpy(), d["Fr"].to_numpy(), B, R)
    F = gaussian_filter(F, sigma=1.2)
    dB, dR = gradient(F, beta_grid, rn_grid)
    curv = gradient(dB, beta_grid, rn_grid)[0] + gradient(dR, beta_grid, rn_grid)[1]
    plt.contour(B, R, curv, levels=[-1e-11, 0, 1e-11], colors=[palette[name]], linewidths=(0.8, 2.0, 0.8))
    cen, lo, hi = bootstrap_centroid(d)
    plt.errorbar(cen[0], cen[1],
                 xerr=[[cen[0]-lo[0]],[hi[0]-cen[0]]],
                 yerr=[[cen[1]-lo[1]],[hi[1]-cen[1]]],
                 fmt='o', color=palette[name], capsize=3, label=f"{name} centroid")

plt.xlabel("β (Sensitivity)"); plt.ylabel("Rₙ (Resilience)")
plt.title("CNT Curvature Overlay — real data appended (smoothed)")
plt.grid(alpha=0.3); plt.xlim(bx); plt.ylim(ry); plt.legend(fontsize=8, loc="best")
plt.show()

# ---------- 3) Update & export phase report ----------
df_all = pd.concat([v.assign(Field=k) for k, v in domains.items()], ignore_index=True)
rows = []
for name, d in domains.items():
    cen, lo, hi = bootstrap_centroid(d)
    rows.append({"Field": name,
                 "beta_mean": cen[0], "beta_lo": lo[0], "beta_hi": hi[0],
                 "Rn_mean": cen[1],   "Rn_lo": lo[1],   "Rn_hi": hi[1],
                 "Fr_mean": float(d["Fr"].mean())})
report = pd.DataFrame(rows).sort_values("beta_mean").reset_index(drop=True)
print("=== CNT Phase Report (with real data) ===")
print(report.to_string(index=False))

# global stability potential
gb, gr = np.linspace(bx[0], bx[1], 160), np.linspace(ry[0], ry[1], 160)
B, R = np.meshgrid(gb, gr)
F = safe_interp(df_all["beta"].to_numpy(), df_all["Rn"].to_numpy(), df_all["Fr"].to_numpy(), B, R)
F = gaussian_filter(F, sigma=1.2)
dB, dR = gradient(F, gb, gr)
curv = gradient(dB, gb, gr)[0] + gradient(dR, gb, gr)[1]
stability_potential = -np.nanmean(curv)
area = np.ptp(gb) * np.ptp(gr)
total_resilience_energy = stability_potential * area
print("\n=== CNT Global Stability Potential (updated) ===")
print(f"Mean curvature (−∇²𝔽ᵣ): {stability_potential:.6e}")
print(f"Phase-plane area:         {area:.6f}")
print(f"Total resilience energy:  {total_resilience_energy:.6e}")

# save
out_dir = "cnt_phase_exports"; os.makedirs(out_dir, exist_ok=True)
report_path = os.path.join(out_dir, "cnt_phase_report_with_real.csv")
fig_path    = os.path.join(out_dir, "cnt_curvature_overlay_with_real.png")
report.to_csv(report_path, index=False); plt.gcf().savefig(fig_path, dpi=180, bbox_inches="tight")
print(f"\nSaved: {report_path}\nSaved: {fig_path}")


In [ ]:
# === CNT More Tests Suite: Surrogates, Noise Ablation, Bootstrap Stability, Sliding Dynamics ===
import numpy as np, matplotlib.pyplot as plt
from numpy.fft import rfft, irfft
from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit
from tqdm import tqdm

# ---------------------- Data selection ----------------------
# Use an existing field matrix if present; else synthesize a resonant one.
X = None
for name in ["X", "signals", "real_data"]:
    if name in globals():
        val = globals()[name]
        if isinstance(val, np.ndarray) and val.ndim == 2:
            X = val; break
if X is None:
    T, N = 1000, 8
    rng = np.random.default_rng(42)
    base = np.sin(np.linspace(0, 10*np.pi, T))[:,None]
    X = base + 0.4*np.sin(np.linspace(0, 6*np.pi, T))[:,None] + 0.12*rng.standard_normal((T,N))

# ---------------------- Helpers ----------------------
def zcorr_entropy(X):
    """Eigenvalue entropy of correlation of z-scored channels."""
    Xs = StandardScaler().fit_transform(X)
    C  = np.corrcoef(Xs.T)
    w, _ = np.linalg.eig(C)
    p = np.abs(w) / np.sum(np.abs(w))
    return -np.sum(p * np.log(p + 1e-12))

def collapse_curve(X, noise_kind="gaussian", steps=np.linspace(0,1,25), rng=None):
    rng = np.random.default_rng(rng)
    T,N = X.shape
    ent = []
    for s in steps:
        if noise_kind == "gaussian":
            Xn = X + rng.standard_normal((T,N))*s
        elif noise_kind == "laplace":
            Xn = X + rng.laplace(0, 1, size=(T,N))*s
        elif noise_kind == "phase":
            # randomize phase of each channel in frequency domain (surrogate-style)
            Xn = np.empty_like(X, dtype=float)
            for j in range(N):
                y = X[:,j]
                Y = rfft(y)
                ang = np.angle(Y)
                mag = np.abs(Y)
                ang_rand = rng.uniform(-np.pi, np.pi, size=ang.shape)
                Ys = mag * np.exp(1j*ang_rand)
                Xn[:,j] = irfft(Ys, n=len(y))
            Xn = (1-s)*X + s*Xn
        else:
            raise ValueError("Unknown noise_kind")
        ent.append(zcorr_entropy(Xn))
    return np.array(ent)

def fit_phase(ent, steps, S0, delta=0.25):
    # collapse threshold where entropy exceeds S0+delta
    idx = np.argmax(ent > S0 + delta)
    gamma_c = steps[idx] if idx>0 else steps[-1]
    Rn = 1.0 - gamma_c
    def S_theo(g, k_r, beta): return S0 + k_r*np.exp(beta*(g - gamma_c))
    (k_r, beta), _ = curve_fit(S_theo, steps, ent, p0=[delta, 1.0], maxfev=10000)
    return dict(gamma_c=float(gamma_c), Rn=float(Rn), k_r=float(k_r), beta=float(beta))

def make_phase_surrogate(X, rng=None):
    """Phase-shuffle each channel → preserves spectrum, destroys phase relations."""
    rng = np.random.default_rng(rng)
    T,N = X.shape
    Xs = np.empty_like(X, dtype=float)
    for j in range(N):
        y = X[:,j]
        Y = rfft(y)
        mag = np.abs(Y)
        ph  = rng.uniform(-np.pi, np.pi, size=Y.shape)
        Ys  = mag * np.exp(1j*ph)
        Xs[:,j] = irfft(Ys, n=len(y))
    return Xs

def recovery_fraction(X):
    """Full collapse (phase-shuffle) then full restoration blend → Fr."""
    Xc = make_phase_surrogate(X, rng=0)
    S0, Sc = zcorr_entropy(X), zcorr_entropy(Xc)
    # full restoration (blend with c=1.0)
    Sf = zcorr_entropy(X)
    Fr = (Sc - Sf) / (Sc - S0 + 1e-12)
    return float(Fr)

# ---------------------- 1) Phase-Shuffle Surrogate Test ----------------------
S_obs = zcorr_entropy(X)
n_surr = 1000
surr_vals = np.zeros(n_surr)
for i in tqdm(range(n_surr), desc="Surrogates"):
    Xs = make_phase_surrogate(X, rng=i+1)
    surr_vals[i] = zcorr_entropy(Xs)
p_phase = np.mean(surr_vals <= S_obs)

plt.figure(figsize=(7,4))
plt.hist(surr_vals, bins=40, edgecolor="k", alpha=0.7)
plt.axvline(S_obs, color="red", lw=2, label=f"Observed S0={S_obs:.3f}")
plt.title(f"Phase-Shuffle Surrogate Test\np={p_phase:.5f}")
plt.xlabel("Entropy under Phase Surrogates"); plt.ylabel("Count"); plt.legend(); plt.show()

# ---------------------- 2) Noise-Model Ablation ----------------------
steps = np.linspace(0,1,25)
S0 = zcorr_entropy(X)
abl = {}
for kind in ["gaussian","laplace","phase"]:
    ent = collapse_curve(X, noise_kind=kind, steps=steps, rng=123)
    pars = fit_phase(ent, steps, S0, delta=0.25)
    abl[kind] = {"ent": ent, **pars}

plt.figure(figsize=(7,5))
for kind, col in zip(["gaussian","laplace","phase"], ["tab:blue","tab:orange","tab:green"]):
    plt.plot(steps, abl[kind]["ent"], "o-", label=f"{kind} (γc={abl[kind]['gamma_c']:.2f}, β={abl[kind]['beta']:.2f})", color=col, alpha=0.9)
plt.axhline(S0, ls="--", color="k", label=f"Base S0={S0:.3f}")
plt.title("Collapse Curves by Noise Model"); plt.xlabel("Noise intensity γ"); plt.ylabel("Entropy S(γ)")
plt.legend(); plt.show()

# Bar comparison for γc and β
labels = list(abl.keys())
gamma_vals = [abl[k]["gamma_c"] for k in labels]
beta_vals  = [abl[k]["beta"]    for k in labels]

plt.figure(figsize=(7,4))
x = np.arange(len(labels))
plt.bar(x-0.2, gamma_vals, width=0.4, label="γc")
plt.bar(x+0.2, beta_vals,  width=0.4, label="β")
plt.xticks(x, labels); plt.ylabel("Value"); plt.title("Noise-Model Ablation: γc vs β"); plt.legend(); plt.show()

# ---------------------- 3) Bootstrap Stability (channels + time) ----------------------
rng = np.random.default_rng(1234)
T,N = X.shape
B = 200  # bootstrap replicates
gamma_bs, beta_bs, Rn_bs, Fr_bs = [], [], [], []

block = max(50, T//10)  # block length for time bootstrap
for b in tqdm(range(B), desc="Bootstrap"):
    # channel bootstrap
    idx_ch = rng.integers(0, N, N)
    # time block bootstrap (concatenate random blocks)
    starts = rng.integers(0, T-block, size=max(1, T//block))
    Xb = np.concatenate([X[s:s+block, idx_ch] for s in starts], axis=0)
    # compute metrics
    S0b = zcorr_entropy(Xb)
    entb = collapse_curve(Xb, noise_kind="gaussian", steps=steps, rng=rng)
    pars = fit_phase(entb, steps, S0b)
    gamma_bs.append(pars["gamma_c"]); beta_bs.append(pars["beta"]); Rn_bs.append(pars["Rn"])
    Fr_bs.append(recovery_fraction(Xb))

def ci(a, q=(0.025, 0.975)): 
    a = np.asarray(a); return np.nanmean(a), np.nanquantile(a, q[0]), np.nanquantile(a, q[1])
g_m, g_lo, g_hi = ci(gamma_bs); b_m, b_lo, b_hi = ci(beta_bs); r_m, r_lo, r_hi = ci(Rn_bs); f_m, f_lo, f_hi = ci(Fr_bs)

print("=== Bootstrap Stability (mean [95% CI]) ===")
print(f"γc : {g_m:.3f} [{g_lo:.3f}, {g_hi:.3f}]")
print(f"β  : {b_m:.3f} [{b_lo:.3f}, {b_hi:.3f}]")
print(f"Rₙ : {r_m:.3f} [{r_lo:.3f}, {r_hi:.3f}]")
print(f"𝔽ᵣ : {f_m:.3f} [{f_lo:.3f}, {f_hi:.3f}]")

plt.figure(figsize=(7,4))
plt.boxplot([gamma_bs, beta_bs, Rn_bs, Fr_bs], labels=["γc","β","Rₙ","𝔽ᵣ"], showmeans=True)
plt.title("Bootstrap Distributions (channels+time)"); plt.grid(alpha=0.3); plt.show()

# ---------------------- 4) Sliding-Window Dynamics (β over time) ----------------------
win = max(200, T//6); hop = max(50, win//4)
starts = np.arange(0, T-win+1, hop)
beta_t = []
for s in starts:
    Xw = X[s:s+win]
    S0w = zcorr_entropy(Xw)
    entw = collapse_curve(Xw, noise_kind="gaussian", steps=steps, rng=0)
    pars = fit_phase(entw, steps, S0w)
    beta_t.append(pars["beta"])
beta_t = np.array(beta_t)

# simple change-point hint via large derivative
d = np.diff(beta_t)
if len(d)>0:
    cp_idx = starts[1:][np.argmax(np.abs(d))]
else:
    cp_idx = None

plt.figure(figsize=(8,4))
plt.plot(starts + win/2, beta_t, "-o")
if cp_idx is not None:
    plt.axvline(cp_idx + win/2, color="red", ls="--", label="max Δβ change")
plt.xlabel("Time (sample index)"); plt.ylabel("β (sensitivity)"); plt.title("Sliding-Window β Dynamics")
if cp_idx is not None: plt.legend()
plt.grid(alpha=0.3); plt.show()

# ---------------------- Summary dict (accessible for programmatic use) ----------------------
summary = {
    "S0_observed": float(S_obs),
    "surrogate_p_value": float(p_phase),
    "ablation": {k: {kk: float(vv) if not isinstance(vv, np.ndarray) else None for kk, vv in abl[k].items()} for k in abl},
    "bootstrap": {
        "gamma_c": {"mean": float(g_m), "lo": float(g_lo), "hi": float(g_hi)},
        "beta":    {"mean": float(b_m), "lo": float(b_lo), "hi": float(b_hi)},
        "R_n":     {"mean": float(r_m), "lo": float(r_lo), "hi": float(r_hi)},
        "F_r":     {"mean": float(f_m), "lo": float(f_lo), "hi": float(f_hi)},
    },
    "sliding_beta": {"centers": (starts + win/2).astype(int).tolist(), "beta": beta_t.tolist(),
                     "max_change_center": int(cp_idx + win/2) if cp_idx is not None else None}
}
print("\n=== CNT More Tests Suite: Summary ===")
for k,v in summary.items(): print(k, ":", v if not isinstance(v, dict) else "(see dict)")


In [ ]:
# === CNT Synchrony-Focused Tests: Circular-Shift, Band-Phase PLV, Lag-Coupling ===
import numpy as np, matplotlib.pyplot as plt
from scipy import signal
from numpy.linalg import eigvals
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# pick the current matrix X (from your notebook); synthesize if absent
X = globals().get("X", globals().get("signals", None))
if X is None:
    T,N = 1000,8
    rng=np.random.default_rng(0)
    t = np.linspace(0,10*np.pi,T)
    X = np.stack([np.sin(t+phi)+0.1*rng.standard_normal(T) for phi in np.linspace(0,2*np.pi,N,endpoint=False)], axis=1)

def eigen_entropy_from_corr(M):
    p = np.abs(eigvals(M))
    p = p / (p.sum() + 1e-12)
    return float(-(p*np.log(p+1e-12)).sum())

def zcorr(X):
    Z = StandardScaler().fit_transform(X)
    return np.corrcoef(Z.T)

# --- (A) Circular-Shift Surrogate on raw correlation entropy ---
def circshift_surrogates_entropy(X, n=1000, rng=1):
    rng = np.random.default_rng(rng)
    T,N = X.shape
    S_obs = eigen_entropy_from_corr(zcorr(X))
    surr = np.empty(n)
    for i in range(n):
        Xs = np.empty_like(X)
        for j in range(N):
            k = rng.integers(0,T)
            Xs[:,j] = np.roll(X[:,j], k)
        surr[i] = eigen_entropy_from_corr(zcorr(Xs))
    p = np.mean(surr <= S_obs)  # low entropy = stronger structure
    return S_obs, surr, p

S_obs, surr, p_circ = circshift_surrogates_entropy(X, n=1000)
plt.figure(figsize=(7,4))
plt.hist(surr, bins=40, edgecolor="k", alpha=0.7)
plt.axvline(S_obs, color="red", lw=2, label=f"Observed={S_obs:.3f}")
plt.title(f"Circular-Shift Surrogates (time alignment destroyed)\np={p_circ:.5f}")
plt.xlabel("Entropy under circular-shifts"); plt.ylabel("Count"); plt.legend(); plt.show()

# --- (B) Band-phase PLV eigen-entropy + circular-shift surrogates ---
def band_hilbert_phase(X, fs=250.0, band=(8,12), order=4):
    # Butter band-pass each channel, take phase via Hilbert
    b,a = signal.butter(order, [band[0]/(fs/2), band[1]/(fs/2)], btype="bandpass")
    Xb = signal.filtfilt(b,a,X,axis=0)
    phase = np.angle(signal.hilbert(Xb, axis=0))
    return phase

def plv_matrix(phase):
    # PLV_ij = |mean(exp(i*(phi_i - phi_j)))|
    N = phase.shape[1]
    PLV = np.ones((N,N))
    for i in range(N):
        for j in range(i+1,N):
            d = np.exp(1j*(phase[:,i]-phase[:,j]))
            v = np.abs(d.mean())
            PLV[i,j]=PLV[j,i]=v
    return PLV

def plv_surrogates_p(X, fs=250.0, band=(8,12), n=500, rng=2):
    phase = band_hilbert_phase(X, fs=fs, band=band)
    S_obs = eigen_entropy_from_corr(plv_matrix(phase))
    rng = np.random.default_rng(rng)
    T,N = X.shape
    surr = np.empty(n)
    for k in range(n):
        # circularly shift phases independently to break synchrony but keep per-channel phase stats
        phase_s = np.empty_like(phase)
        for j in range(N):
            shift = rng.integers(0,T)
            phase_s[:,j] = np.roll(phase[:,j], shift)
        surr[k] = eigen_entropy_from_corr(plv_matrix(phase_s))
    p = np.mean(surr <= S_obs)
    return S_obs, surr, p

# estimate a sampling rate if unknown (assume 250 Hz for demo)
fs_guess = 250.0
S_plv, s_plv, p_plv = plv_surrogates_p(X, fs=fs_guess, band=(8,12), n=500)
plt.figure(figsize=(7,4))
plt.hist(s_plv, bins=40, edgecolor="k", alpha=0.7)
plt.axvline(S_plv, color="red", lw=2, label=f"Observed={S_plv:.3f}")
plt.title(f"Band-Phase PLV (8–12 Hz) Circular-Shift Surrogates\np={p_plv:.5f}")
plt.xlabel("PLV eigen-entropy (surrogates)"); plt.ylabel("Count"); plt.legend(); plt.show()

# --- (C) Max-lag coupling (up to L) eigen-entropy + circular-shifts ---
def maxlag_corr_matrix(X, L=50):
    # compute channel-channel max absolute cross-corr over lags [-L..L]
    T,N = X.shape
    Z = StandardScaler().fit_transform(X)
    M = np.ones((N,N))
    for i in range(N):
        for j in range(i+1,N):
            xi, xj = Z[:,i], Z[:,j]
            vals=[]
            for lag in range(-L,L+1):
                if lag>=0:
                    v = np.corrcoef(xi[lag:], xj[:T-lag])[0,1]
                else:
                    v = np.corrcoef(xi[:T+lag], xj[-lag:])[0,1]
                vals.append(np.abs(v))
            m = np.max(vals)
            M[i,j]=M[j,i]=m
    return M

def maxlag_surrogates_p(X, L=50, n=300, rng=3):
    S_obs = eigen_entropy_from_corr(maxlag_corr_matrix(X, L=L))
    rng = np.random.default_rng(rng)
    T,N = X.shape
    surr = np.empty(n)
    for k in range(n):
        Xs = np.empty_like(X)
        for j in range(N):
            shift = rng.integers(0,T)
            Xs[:,j] = np.roll(X[:,j], shift)
        surr[k] = eigen_entropy_from_corr(maxlag_corr_matrix(Xs, L=L))
    p = np.mean(surr <= S_obs)
    return S_obs, surr, p

S_lag, s_lag, p_lag = maxlag_surrogates_p(X, L=50, n=300)
plt.figure(figsize=(7,4))
plt.hist(s_lag, bins=40, edgecolor="k", alpha=0.7)
plt.axvline(S_lag, color="red", lw=2, label=f"Observed={S_lag:.3f}")
plt.title(f"Max-Lag Coupling (L=50) Circular-Shift Surrogates\np={p_lag:.5f}")
plt.xlabel("Max-lag eigen-entropy (surrogates)"); plt.ylabel("Count"); plt.legend(); plt.show()

print("=== Synchrony-Focused p-values ===")
print(f"(A) Raw corr (circular shifts):  p = {p_circ:.5f}")
print(f"(B) Band-phase PLV (8–12 Hz):    p = {p_plv:.5f}")
print(f"(C) Max-lag coupling (L=50):     p = {p_lag:.5f}")


In [ ]:
# === CNT Synchrony Escalation: Multi-Band PLV + Coherence with IAAFT Surrogates & FDR ===
import numpy as np, matplotlib.pyplot as plt
from scipy import signal
from numpy.linalg import eigvals
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests

# ---------- data ----------
X = globals().get("X", globals().get("signals", None))
if X is None:
    T,N = 3000, 12
    rng = np.random.default_rng(0)
    t = np.linspace(0, 30, T)
    base = np.sin(2*np.pi*10*t)[:,None]           # alpha-ish
    X = base + 0.35*np.sin(2*np.pi*22*t)[:,None]  # beta-ish
    X += 0.15*rng.standard_normal((T,N))

fs = globals().get("fs", 250.0)   # set your sampling rate if you have it

# ---------- helpers ----------
def iaaft_surrogate(x, iters=200, rng=None):
    """Univariate IAAFT surrogate (preserves spectrum & amplitude distribution)."""
    rng = np.random.default_rng(rng)
    s = np.sort(x)
    y = rng.permutation(x)
    target_mag = np.abs(np.fft.rfft(x))
    for _ in range(iters):
        Y = np.fft.rfft(y)
        y = np.fft.irfft(target_mag * np.exp(1j*np.angle(Y)), n=len(x)).real
        # rank-order match to original distribution
        ranks = np.argsort(np.argsort(y))
        y = s[ranks]
    return y

def iaaft_matrix(X, iters=200, rng=None):
    rng = np.random.default_rng(rng)
    return np.stack([iaaft_surrogate(X[:,j], iters=iters, rng=rng) for j in range(X.shape[1])], axis=1)

def eigen_entropy(M):
    w = np.abs(eigvals(M)); w = w/(w.sum()+1e-12)
    return float(-(w*np.log(w+1e-12)).sum())

def zcorr(X):
    Z = StandardScaler().fit_transform(X)
    return np.corrcoef(Z.T)

def band_pass(X, fs, f_lo, f_hi, order=4):
    b,a = signal.butter(order, [f_lo/(fs/2), f_hi/(fs/2)], btype='band')
    return signal.filtfilt(b,a,X,axis=0)

def plv_entropy(Xb):
    phase = np.angle(signal.hilbert(Xb, axis=0))
    N = phase.shape[1]
    PLV = np.ones((N,N))
    for i in range(N):
        for j in range(i+1,N):
            d = np.exp(1j*(phase[:,i]-phase[:,j]))
            PLV[i,j]=PLV[j,i]=np.abs(d.mean())
    return eigen_entropy(PLV)

def coh_entropy(Xb, fs, nper=512, nover=256, multitaper=False):
    N=Xb.shape[1]
    COH = np.ones((N,N))
    for i in range(N):
        for j in range(i+1,N):
            f, Cxy = signal.coherence(Xb[:,i], Xb[:,j], fs=fs, nperseg=nper, noverlap=nover)
            COH[i,j]=COH[j,i]=np.mean(Cxy)
    return eigen_entropy(COH)

# ---------- bands, surrogates, tests ----------
bands = [("δ",1,4),("θ",4,8),("α",8,12),("β",13,30),("γ",30,45)]
n_surr = 400
plv_p, coh_p = [], []

for label, f1, f2 in bands:
    Xb = band_pass(X, fs, f1, f2)
    S_plv_obs = plv_entropy(Xb)
    S_coh_obs = coh_entropy(Xb, fs)

    surr_plv = np.zeros(n_surr)
    surr_coh = np.zeros(n_surr)
    for k in range(n_surr):
        Xs = iaaft_matrix(Xb, iters=80, rng=k+1)   # strict null
        surr_plv[k] = plv_entropy(Xs)
        surr_coh[k] = coh_entropy(Xs, fs)

    p_plv = np.mean(surr_plv <= S_plv_obs)
    p_coh = np.mean(surr_coh <= S_coh_obs)
    plv_p.append(p_plv); coh_p.append(p_coh)

plv_p = np.array(plv_p); coh_p = np.array(coh_p)

# FDR across all tests
_, plv_q, _, _ = multipletests(plv_p, alpha=0.05, method='fdr_bh')
_, coh_q, _, _ = multipletests(coh_p, alpha=0.05, method='fdr_bh')

# ---------- plot ----------
fig, axes = plt.subplots(1,2, figsize=(10,4))
axes[0].bar(range(len(bands)), -np.log10(plv_q+1e-12))
axes[0].set_xticks(range(len(bands))); axes[0].set_xticklabels([b[0] for b in bands])
axes[0].set_ylabel("-log10(q)"); axes[0].set_title("PLV eigen-entropy vs IAAFT (FDR-q)")

axes[1].bar(range(len(bands)), -np.log10(coh_q+1e-12))
axes[1].set_xticks(range(len(bands))); axes[1].set_xticklabels([b[0] for b in bands])
axes[1].set_ylabel("-log10(q)"); axes[1].set_title("Coherence eigen-entropy vs IAAFT (FDR-q)")
for ax in axes:
    ax.axhline(-np.log10(0.05), color='r', ls='--', lw=1, label='q=0.05'); ax.legend()
plt.tight_layout(); plt.show()

print("Bands:", [b[0] for b in bands])
print("PLV  q-values:", np.round(plv_q,5))
print("COH  q-values:", np.round(coh_q,5))


In [ ]:
# === Leakage-Robust Connectivity Battery (iCoh / AEC-orth / Partial Coherence) ===
import numpy as np, matplotlib.pyplot as plt
from scipy import signal, linalg
from numpy.linalg import eigvals
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests

# ---- USE CURRENT FIELD ----
X = globals().get("X", globals().get("signals"))
fs = float(globals().get("fs", 250.0))  # set your fs if known

# ---- CONFIG (FAST) ----
bands = [("δ",1,4),("θ",4,8),("α",8,12),("β",13,30),("γ",30,45)]
n_surr = 120          # ↑ for slower / stronger
iaaft_iters = 40      # ↑ for stricter nulls
welch_nper = 256; welch_nover = 128

# ---- HELPERS ----
def band_pass(X, fs, f1, f2, order=4):
    b,a = signal.butter(order, [f1/(fs/2), f2/(fs/2)], btype='band')
    return signal.filtfilt(b,a,X,axis=0)

def iaaft_surrogate(x, iters=100, rng=None):
    rng = np.random.default_rng(rng); s = np.sort(x); y = rng.permutation(x)
    target_mag = np.abs(np.fft.rfft(x))
    for _ in range(iters):
        Y = np.fft.rfft(y)
        y = np.fft.irfft(target_mag*np.exp(1j*np.angle(Y)), n=len(x)).real
        ranks = np.argsort(np.argsort(y)); y = s[ranks]
    return y

def iaaft_matrix(X, iters=80, rng=None):
    rng = np.random.default_rng(rng)
    return np.stack([iaaft_surrogate(X[:,j], iters=iters, rng=rng) for j in range(X.shape[1])], axis=1)

def mean_imag_coh(Xb, fs, nper=256, nover=128):
    """Return NxN imaginary coherence matrix averaged over band."""
    N = Xb.shape[1]; iC = np.ones((N,N))
    for i in range(N):
        for j in range(i+1,N):
            f, Pxy = signal.csd(Xb[:,i], Xb[:,j], fs=fs, nperseg=nper, noverlap=nover)
            _, Pxx = signal.welch(Xb[:,i], fs=fs, nperseg=nper, noverlap=nover)
            _, Pyy = signal.welch(Xb[:,j], fs=fs, nperseg=nper, noverlap=nover)
            Cxy = Pxy/np.sqrt(Pxx*Pyy + 1e-12)
            imC = np.abs(np.imag(Cxy))
            iC[i,j]=iC[j,i]=np.nanmean(imC)
    return iC

def aec_orth(Xb):
    """Orthogonalized amplitude-envelope correlation (Colclough+)."""
    # Hilbert envelopes
    Z = signal.hilbert(Xb, axis=0)
    N = Z.shape[1]; A = np.abs(Z)  # envelopes
    # symmetric orthogonalization (remove instantaneous linear mixing)
    U, s, Vt = linalg.svd(A, full_matrices=False)
    A_orth = U @ Vt
    R = np.corrcoef(A_orth.T)
    return R

def partial_coherence(Xb, fs, nper=256, nover=128):
    """Partial coherence via inverse spectral density (precision)."""
    N = Xb.shape[1]
    # average spectral density matrix across freqs in band
    S_sum = np.zeros((N,N), dtype=np.complex128); count=0
    for k in range(N):
        for l in range(N):
            f, Pkl = signal.csd(Xb[:,k], Xb[:,l], fs=fs, nperseg=nper, noverlap=nover)
            if k==0 and l==0: freqs = f
            S_sum[k,l] = np.mean(Pkl)
    S = S_sum
    # precision matrix of spectrum
    P = linalg.pinvh(S + 1e-9*np.eye(N))
    # partial coherence magnitude
    pc = np.zeros((N,N))
    for i in range(N):
        for j in range(i+1,N):
            num = np.abs(P[i,j])**2
            den = P[i,i]*P[j,j]
            pc[i,j]=pc[j,i]=float(np.real(num/den))
    return pc

def eig_entropy(M):
    w = np.abs(eigvals(M)); w = w/(w.sum()+1e-12)
    return float(-(w*np.log(w+1e-12)).sum())

# ---- MAIN LOOP ----
res = []
for label,f1,f2 in bands:
    Xb = band_pass(X, fs, f1, f2)
    # observed
    S_iC  = eig_entropy(mean_imag_coh(Xb, fs, welch_nper, welch_nover))
    S_AEC = eig_entropy(aec_orth(Xb))
    S_PC  = eig_entropy(partial_coherence(Xb, fs, welch_nper, welch_nover))
    # surrogates
    s_iC, s_AEC, s_PC = [], [], []
    for k in range(n_surr):
        Xs = iaaft_matrix(Xb, iters=iaaft_iters, rng=k+1)
        s_iC.append(eig_entropy(mean_imag_coh(Xs, fs, welch_nper, welch_nover)))
        s_AEC.append(eig_entropy(aec_orth(Xs)))
        s_PC.append(eig_entropy(partial_coherence(Xs, fs, welch_nper, welch_nover)))
    p_iC  = np.mean(np.array(s_iC)  <= S_iC)
    p_AEC = np.mean(np.array(s_AEC) <= S_AEC)
    p_PC  = np.mean(np.array(s_PC)  <= S_PC)
    res.append((label, p_iC, p_AEC, p_PC, S_iC, S_AEC, S_PC))

labels, p_iC, p_AEC, p_PC, S_iC, S_AEC, S_PC = map(np.array, zip(*res))

# FDR across all tests (3 per band)
from numpy import concatenate
all_p = concatenate([p_iC, p_AEC, p_PC])
rej, q_all, _, _ = multipletests(all_p, alpha=0.05, method='fdr_bh')
q_iC, q_AEC, q_PC = q_all[:len(bands)], q_all[len(bands):2*len(bands)], q_all[2*len(bands):]

# ---- PLOTS ----
fig, axes = plt.subplots(1,3, figsize=(12,4))
for ax, q, ttl in zip(axes,
                      [q_iC, q_AEC, q_PC],
                      ["Imaginary Coherence (iCoh)", "AEC-orth", "Partial Coherence"]):
    ax.bar(range(len(bands)), -np.log10(q+1e-12))
    ax.set_xticks(range(len(bands))); ax.set_xticklabels(labels)
    ax.axhline(-np.log10(0.05), color='r', ls='--', lw=1, label='q=0.05')
    ax.set_ylabel("-log10(q)"); ax.set_title(ttl); ax.legend()
plt.tight_layout(); plt.show()

print("Bands:", list(labels))
print("q_iCoh:", np.round(q_iC,5))
print("q_AEC :", np.round(q_AEC,5))
print("q_PartialCoh:", np.round(q_PC,5))


In [ ]:
# === CNT Consensus Information-Flow Graph (wPLI + PSI + Granger across bands) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import signal
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.multitest import multipletests
import os

# ---------------------- CONFIG ----------------------
# If you already defined X (T×N) and fs earlier, this cell will use them.
X  = globals().get("X", globals().get("signals"))
fs = float(globals().get("fs", 250.0))  # set your true sampling rate if known

# Bands to scan:
bands = [("δ",1,4),("θ",4,8),("α",8,12),("β",13,30),("γ",30,45)]

# Speed/strictness knobs:
N_KEEP        = min(X.shape[1], 16)     # limit channels to top-left N (set to X.shape[1] to keep all)
N_PER, N_OVER = 512, 256                # Welch params
VAR_MAX_LAG   = 10                      # VAR order search upper bound
GRANGER_Q     = 0.10                    # lenient screen for edges per band
EXPORT        = True                    # save CSV/PNGs to disk

# ---------------------- HELPERS ----------------------
def band_pass(X, fs, f1, f2, order=4):
    b,a = signal.butter(order, [f1/(fs/2), f2/(fs/2)], btype='band')
    return signal.filtfilt(b,a,X,axis=0)

def wpli_debiased(x, y, fs, nper=512, nover=256):
    f, Pxy = signal.csd(x, y, fs=fs, nperseg=nper, noverlap=nover)
    _, Pxx = signal.welch(x, fs=fs, nperseg=nper, noverlap=nover)
    _, Pyy = signal.welch(y, fs=fs, nperseg=nper, noverlap=nover)
    Cxy = Pxy/np.sqrt(Pxx*Pyy + 1e-12)
    im = np.imag(Cxy)
    num = (np.abs(im).mean()**2 - (im**2).mean())
    den = (1 - (im**2).mean())
    return float(np.clip(num/(den+1e-12), 0, 1))

def psi(x, y, fs, band, nper=512, nover=256):
    f, Pxy = signal.csd(x, y, fs=fs, nperseg=nper, noverlap=nover)
    mask = (f>=band[0]) & (f<=band[1])
    C = Pxy[mask] / (np.abs(Pxy[mask]) + 1e-12)
    val = 0.0
    for k in range(len(C)-1):
        val += np.imag(np.conj(C[k]) * C[k+1])
    return float(val)

def granger_xy(x, y, max_lag=10):
    Z = np.vstack([x, y]).T
    Z = Z - Z.mean(0)
    best_aic, best = np.inf, (1.0, 1.0)
    for p in range(1, max_lag+1):
        try:
            m = VAR(Z).fit(p)
            if m.aic < best_aic:
                best_aic = m.aic
                fx = m.test_causality(0, [1], kind='f')  # y -> x
                fy = m.test_causality(1, [0], kind='f')  # x -> y
                best = (fx.pvalue, fy.pvalue)
        except Exception:
            continue
    return best  # (p_yx, p_xy)

# ---------------------- PREP ----------------------
X = X[:, :N_KEEP]  # optional downselect
T, N = X.shape
labels = [f"ch{j}" for j in range(N)]
os.makedirs("cnt_flow_exports", exist_ok=True)

# ---------------------- PER-BAND ANALYSIS ----------------------
all_band_edges = []
for label, f1, f2 in bands:
    Xb = band_pass(X, fs, f1, f2)
    wpli = np.zeros((N,N)); psi_mat = np.zeros((N,N))
    p_yx = np.ones((N,N));  p_xy = np.ones((N,N))

    for i in range(N):
        for j in range(i+1, N):
            w = wpli_debiased(Xb[:,i], Xb[:,j], fs, N_PER, N_OVER)
            wpli[i,j]=wpli[j,i]=w
            ps = psi(Xb[:,i], Xb[:,j], fs, (f1,f2), N_PER, N_OVER)
            psi_mat[i,j]= ps; psi_mat[j,i]= -ps
            pyx, pxy = granger_xy(Xb[:,i], Xb[:,j], max_lag=VAR_MAX_LAG)
            p_yx[i,j]=pyx; p_xy[j,i]=pxy

    # FDR over both directions for all pairs
    tri = np.triu_indices(N,1)
    pg = np.concatenate([p_yx[tri], p_xy[tri]])
    rej, q_all, _, _ = multipletests(pg, alpha=0.05, method='fdr_bh')
    q_dir = np.full((N,N), 1.0)
    m = len(p_yx[tri])
    q_dir[tri] = q_all[:m]             # y->x on upper triangle
    temp = np.full((N,N), 1.0); temp[tri] = q_all[m:]  # x->y on upper
    q_dir = np.minimum(q_dir, temp.T)  # place both directions

    # edge table for this band
    rows=[]
    for i in range(N):
        for j in range(N):
            if i==j: continue
            score = 0.6*wpli[i,j] + 0.4*abs(psi_mat[i,j])
            rows.append({"band": label, "src": labels[i], "dst": labels[j],
                         "wPLI": wpli[i,j], "PSI": psi_mat[i,j],
                         "q_Granger": q_dir[i,j], "score": score})
    band_df = pd.DataFrame(rows)
    band_df["keep"] = (band_df["q_Granger"] < GRANGER_Q)
    all_band_edges.append(band_df.sort_values("score", ascending=False))

# ---------------------- CONSENSUS GRAPH ----------------------
# Stack and summarize: how many bands kept each edge? what's the mean score?
stack = pd.concat(all_band_edges, ignore_index=True)
cons = (stack[stack["keep"]]
        .groupby(["src","dst"], as_index=False)
        .agg(bands_present=("band","count"),
             mean_score=("score","mean"),
             bands_list=("band", lambda b: ",".join(sorted(map(str,set(b)))))))
cons = cons.sort_values(["bands_present","mean_score"], ascending=[False, False]).reset_index(drop=True)

print("=== Consensus edges (q_Granger < {:.2f} in at least one band) ===".format(GRANGER_Q))
display_cols = ["src","dst","bands_present","mean_score","bands_list"]
print(cons[display_cols].head(20).to_string(index=False))

# ---------------------- VISUALS ----------------------
# 1) Consensus adjacency heatmap (weight = bands_present * mean_score)
W = np.zeros((N,N))
for _, row in cons.iterrows():
    i = labels.index(row["src"]); j = labels.index(row["dst"])
    W[i,j] = row["bands_present"] * row["mean_score"]

plt.figure(figsize=(7,6))
plt.imshow(W, cmap="inferno")
plt.colorbar(label="Consensus weight (bands×score)")
plt.xticks(range(N), labels, rotation=90)
plt.yticks(range(N), labels)
plt.title("CNT Consensus Directed Connectivity (multi-band)")
plt.tight_layout()
if EXPORT: plt.savefig("cnt_flow_exports/consensus_adjacency.png", dpi=180)
plt.show()

# 2) Per-band edge count bar plot
counts = stack.groupby("band")["keep"].sum().reindex([b[0] for b in bands])
plt.figure(figsize=(7,3))
counts.plot(kind="bar")
plt.ylabel("# edges (q<{:.2f})".format(GRANGER_Q))
plt.title("Edges surviving per band")
plt.tight_layout()
if EXPORT: plt.savefig("cnt_flow_exports/edges_per_band.png", dpi=180)
plt.show()

# 3) Save CSVs
if EXPORT:
    stack.to_csv("cnt_flow_exports/per_band_edges.csv", index=False)
    cons.to_csv("cnt_flow_exports/consensus_edges.csv", index=False)
    print("Saved CSVs and PNGs to: cnt_flow_exports/")


In [ ]:
# === CNT Edge Robustness Battery: split-half, time-reversal, block-bootstrap, lag-jitter ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.multitest import multipletests
from scipy import signal
import os

# Use your current X (T×N), fs, and the consensus edge table from the last step.
X  = globals().get("X", globals().get("signals"))
fs = float(globals().get("fs", 250.0))
cons_path = "cnt_flow_exports/consensus_edges.csv"
cons = pd.read_csv(cons_path)

# --- helpers ---
def band_pass(X, fs, f1, f2, order=4):
    b,a = signal.butter(order, [f1/(fs/2), f2/(fs/2)], btype='band')
    return signal.filtfilt(b,a,X,axis=0)

def granger_xy(x, y, max_lag=10):
    Z = np.vstack([x, y]).T - np.mean(np.vstack([x, y]).T, axis=0)
    best_aic, best = np.inf, (1.0, 1.0)
    for p in range(1, max_lag+1):
        try:
            m = VAR(Z).fit(p)
            if m.aic < best_aic:
                best_aic = m.aic
                fx = m.test_causality(0, [1], kind='f')  # y -> x
                fy = m.test_causality(1, [0], kind='f')  # x -> y
                best = (fx.pvalue, fy.pvalue)
        except Exception:
            continue
    return best  # (p_yx, p_xy)

def edge_pvals(Xb, src_idx, dst_idx, max_lag=10):
    p_yx, p_xy = granger_xy(Xb[:,src_idx], Xb[:,dst_idx], max_lag=max_lag)
    # q via FDR over both directions for this pair only (trivial but consistent)
    rej, q, _, _ = multipletests([p_yx, p_xy], alpha=0.05, method="fdr_bh")
    return float(q[0]), float(q[1])  # q(src->dst), q(dst->src)

# --- config ---
bands = [("δ",1,4),("θ",4,8),("α",8,12),("β",13,30),("γ",30,45)]
MAX_LAG = 10
B_BOOT  = 200     # block-bootstrap reps
BLOCK   = max(50, X.shape[0]//10)
LAG_JIT = 3       # samples of random lag jitter
EXPORT_DIR = "cnt_flow_exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

# Build label->index map
labels = [f"ch{j}" for j in range(X.shape[1])]
idx = {lab:i for i,lab in enumerate(labels)}

rows=[]
for k, row in cons.iterrows():
    s = idx[row["src"]]; d = idx[row["dst"]]
    bands_here = str(row["bands_list"]).split(",")
    # 1) split-half stability (across bands where edge existed)
    sh_hits, sh_total = 0, 0
    tr_hits, bb_score, lj_score = 0, 0.0, 0.0
    for lab,f1,f2 in bands:
        if lab not in bands_here: 
            continue
        Xb = band_pass(X, fs, f1, f2)
        T = Xb.shape[0]
        A, B = Xb[:T//2], Xb[T//2:]
        # split-half q
        qAB = []
        for part in (A,B):
            q_sd, _ = edge_pvals(part, s, d, MAX_LAG)
            qAB.append(q_sd)
        sh_total += 2
        sh_hits  += sum(q < 0.10 for q in qAB)   # lenient q<0.10
        # 2) time-reversal sanity (should weaken/flip)
        Xrev = Xb[::-1].copy()
        q_forward, _ = edge_pvals(Xb,   s, d, MAX_LAG)
        q_reverse, _ = edge_pvals(Xrev, s, d, MAX_LAG)
        tr_hits += 1 if (q_forward < 0.10 and q_reverse > 0.10) else 0
        # 3) block-bootstrap persistence
        rng = np.random.default_rng(0)
        keep = 0
        for _ in range(B_BOOT):
            starts = rng.integers(0, T-BLOCK, size=max(1, T//BLOCK))
            Xbb = np.concatenate([Xb[st:st+BLOCK] for st in starts], axis=0)
            q_sd, _ = edge_pvals(Xbb, s, d, MAX_LAG)
            keep += (q_sd < 0.10)
        bb_score += keep / B_BOOT
        # 4) lag-jitter robustness
        keep_lj = 0
        for _ in range(100):
            jitter = np.random.randint(-LAG_JIT, LAG_JIT+1)
            Xj = Xb.copy()
            Xj[:,d] = np.roll(Xj[:,d], jitter)
            q_sd, _ = edge_pvals(Xj, s, d, MAX_LAG)
            keep_lj += (q_sd < 0.10)
        lj_score += keep_lj / 100.0

    # normalize to 0..100
    split_half_pct = 100.0 * (sh_hits / max(1, sh_total))
    time_rev_pct   = 100.0 * (tr_hits  / max(1, len(bands_here)))
    bb_pct         = 100.0 * (bb_score / max(1, len(bands_here)))
    lj_pct         = 100.0 * (lj_score / max(1, len(bands_here)))
    # composite robustness
    R = 0.35*split_half_pct + 0.25*time_rev_pct + 0.25*bb_pct + 0.15*lj_pct
    rows.append({
        "src": row["src"], "dst": row["dst"], "bands": row["bands_list"],
        "split_half%": split_half_pct, "time_reverse%": time_rev_pct,
        "bootstrap%": bb_pct, "lag_jitter%": lj_pct,
        "robustness_score": R
    })

rob = pd.DataFrame(rows).sort_values("robustness_score", ascending=False).reset_index(drop=True)
print("=== CNT Edge Robustness (0–100) ===")
print(rob.head(20).to_string(index=False))

# Plot top-12 radar-like bars
top = rob.head(12).copy()
plt.figure(figsize=(9,4.2))
for i,(name,row) in enumerate(top.iterrows()):
    plt.bar(i-0.45, row["split_half%"], width=0.3, label="split-half" if i==0 else None)
    plt.bar(i-0.15, row["time_reverse%"], width=0.3, label="time-reverse" if i==0 else None)
    plt.bar(i+0.15, row["bootstrap%"],   width=0.3, label="bootstrap"   if i==0 else None)
    plt.bar(i+0.45, row["lag_jitter%"],  width=0.3, label="lag-jitter"  if i==0 else None)
plt.xticks(range(len(top)), [f"{r['src']}→{r['dst']}" for _,r in top.iterrows()], rotation=45, ha="right")
plt.ylabel("Pass rate (%)"); plt.title("Top Edge Robustness Components")
plt.legend(ncol=4, loc="upper center", bbox_to_anchor=(0.5,1.22))
plt.tight_layout()
plt.savefig(f"{EXPORT_DIR}/edge_robustness_components.png", dpi=180)
plt.show()

# Save table
rob_path = f"{EXPORT_DIR}/edge_robustness_scores.csv"
rob.to_csv(rob_path, index=False)
print(f"Saved: {rob_path}\nSaved: {EXPORT_DIR}/edge_robustness_components.png")


In [ ]:
# === Self-healing Edge Robustness Battery (defines X/fs if missing) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.multitest import multipletests
from scipy import signal
import os

# ---------- 0) Locate or synthesize field ----------
def resolve_field():
    cand = [globals().get(k) for k in ["X","signals","real_data"]]
    for a in cand:
        if isinstance(a, np.ndarray) and a.ndim==2:
            return a
    # synth fallback
    T,N = 2000, 8
    rng = np.random.default_rng(7)
    t = np.linspace(0, 20, T)
    base = np.sin(2*np.pi*10*t)[:,None]
    Xsyn = base + 0.25*np.sin(2*np.pi*22*t)[:,None] + 0.12*rng.standard_normal((T,N))
    return Xsyn

def resolve_fs():
    v = globals().get("fs", None)
    try:
        return float(v) if v is not None else 250.0
    except Exception:
        return 250.0

X  = resolve_field()
fs = resolve_fs()

# ---------- 1) Load consensus edges ----------
CONS_PATH = "cnt_flow_exports/consensus_edges.csv"
if not os.path.exists(CONS_PATH):
    raise FileNotFoundError("Expected consensus_edges.csv from the prior step. Re-run the consensus flow cell first.")
cons = pd.read_csv(CONS_PATH)

# ---------- 2) Helpers ----------
def band_pass(X, fs, f1, f2, order=4):
    b,a = signal.butter(order, [f1/(fs/2), f2/(fs/2)], btype='band')
    return signal.filtfilt(b,a,X,axis=0)

def granger_xy(x, y, max_lag=10):
    Z = np.vstack([x, y]).T
    Z = Z - Z.mean(0)
    best_aic, best = np.inf, (1.0, 1.0)
    for p in range(1, max_lag+1):
        try:
            m = VAR(Z).fit(p)
            if m.aic < best_aic:
                best_aic = m.aic
                fx = m.test_causality(0, [1], kind='f')  # y -> x ?
                fy = m.test_causality(1, [0], kind='f')  # x -> y ?
                best = (fx.pvalue, fy.pvalue)
        except Exception:
            continue
    return best

def edge_q_forward(Xb, s, d, max_lag=10):
    p_yx, p_xy = granger_xy(Xb[:,s], Xb[:,d], max_lag=max_lag)
    q = multipletests([p_yx, p_xy], alpha=0.05, method="fdr_bh")[1][0]
    return float(q)

# ---------- 3) Config ----------
bands = [("δ",1,4),("θ",4,8),("α",8,12),("β",13,30),("γ",30,45)]
MAX_LAG = 10
B_BOOT  = 200
BLOCK   = max(50, X.shape[0]//10)
LAG_JIT = 3
EXPORT_DIR = "cnt_flow_exports"; os.makedirs(EXPORT_DIR, exist_ok=True)

labels = [f"ch{j}" for j in range(X.shape[1])]
idx = {lab:i for i,lab in enumerate(labels)}

# ---------- 4) Robustness battery ----------
rows=[]
for _, row in cons.iterrows():
    s = idx[row["src"]]; d = idx[row["dst"]]
    bands_here = str(row["bands_list"]).split(",")

    sh_hits = sh_total = tr_hits = 0
    bb_score = lj_score = 0.0

    for lab,f1,f2 in bands:
        if lab not in bands_here: 
            continue
        Xb = band_pass(X, fs, f1, f2)
        T = Xb.shape[0]

        # split-half
        partA, partB = Xb[:T//2], Xb[T//2:]
        for part in (partA, partB):
            q = edge_q_forward(part, s, d, MAX_LAG)
            sh_total += 1; sh_hits += (q < 0.10)

        # time-reversal sanity
        q_fwd = edge_q_forward(Xb, s, d, MAX_LAG)
        Xrev  = Xb[::-1].copy()
        q_rev = edge_q_forward(Xrev, s, d, MAX_LAG)
        tr_hits += 1 if (q_fwd < 0.10 and q_rev > 0.10) else 0

        # block-bootstrap persistence
        rng = np.random.default_rng(0)
        keep = 0
        for _ in range(B_BOOT):
            starts = rng.integers(0, T-BLOCK, size=max(1, T//BLOCK))
            Xbb = np.concatenate([Xb[st:st+BLOCK] for st in starts], axis=0)
            q = edge_q_forward(Xbb, s, d, MAX_LAG)
            keep += (q < 0.10)
        bb_score += keep / B_BOOT

        # lag-jitter robustness
        keep_lj = 0
        for _ in range(100):
            jitter = np.random.randint(-LAG_JIT, LAG_JIT+1)
            Xj = Xb.copy(); Xj[:,d] = np.roll(Xj[:,d], jitter)
            q = edge_q_forward(Xj, s, d, MAX_LAG)
            keep_lj += (q < 0.10)
        lj_score += keep_lj / 100.0

    # scores (0–100)
    split_half_pct = 100.0 * (sh_hits / max(1, sh_total))
    time_rev_pct   = 100.0 * (tr_hits  / max(1, len([b for b in bands_here if b!=''])))
    bb_pct         = 100.0 * (bb_score / max(1, len([b for b in bands_here if b!=''])))
    lj_pct         = 100.0 * (lj_score / max(1, len([b for b in bands_here if b!=''])))

    R = 0.35*split_half_pct + 0.25*time_rev_pct + 0.25*bb_pct + 0.15*lj_pct
    rows.append({
        "src": row["src"], "dst": row["dst"], "bands": row["bands_list"],
        "split_half%": split_half_pct, "time_reverse%": time_rev_pct,
        "bootstrap%": bb_pct, "lag_jitter%": lj_pct,
        "robustness_score": R
    })

rob = pd.DataFrame(rows).sort_values("robustness_score", ascending=False).reset_index(drop=True)
print("=== CNT Edge Robustness (0–100) ===")
print(rob.head(20).to_string(index=False))

# visualize top-12 components
top = rob.head(12).copy()
plt.figure(figsize=(9,4.2))
for i,(_,r) in enumerate(top.iterrows()):
    plt.bar(i-0.45, r["split_half%"], width=0.3, label="split-half" if i==0 else None)
    plt.bar(i-0.15, r["time_reverse%"], width=0.3, label="time-reverse" if i==0 else None)
    plt.bar(i+0.15, r["bootstrap%"],   width=0.3, label="bootstrap"   if i==0 else None)
    plt.bar(i+0.45, r["lag_jitter%"],  width=0.3, label="lag-jitter"  if i==0 else None)
plt.xticks(range(len(top)), [f"{r['src']}→{r['dst']}" for _,r in top.iterrows()], rotation=45, ha="right")
plt.ylabel("Pass rate (%)"); plt.title("Top Edge Robustness Components")
plt.legend(ncol=4, loc="upper center", bbox_to_anchor=(0.5,1.22))
plt.tight_layout()
plt.savefig(f"{EXPORT_DIR}/edge_robustness_components.png", dpi=180)
plt.show()

rob_path = f"{EXPORT_DIR}/edge_robustness_scores.csv"
rob.to_csv(rob_path, index=False)
print(f"Saved: {rob_path}\nSaved: {EXPORT_DIR}/edge_robustness_components.png")


In [ ]:
# === CNT Robustness: Diagnostic + Quick Run (always prints) ===
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.multitest import multipletests
from scipy import signal

def _resolve_field():
    for k in ["X","signals","real_data"]:
        v = globals().get(k, None)
        if isinstance(v, np.ndarray) and v.ndim==2:
            print(f"[ok] Using field from variable: {k}  shape={v.shape}")
            return v
    # synth fallback (guarantees output)
    T,N = 2000,8
    print("[warn] No X/signals/real_data found → using synthetic fallback "
          f"(T={T}, N={N})")
    t = np.linspace(0,20,T)
    rng = np.random.default_rng(7)
    X = np.sin(2*np.pi*10*t)[:,None] + 0.25*np.sin(2*np.pi*22*t)[:,None] + 0.12*rng.standard_normal((T,N))
    return X

def _resolve_fs():
    v = globals().get("fs", None)
    if v is None:
        print("[warn] No fs found → defaulting to 250.0 Hz")
        return 250.0
    try:
        f = float(v)
        print(f"[ok] Using fs={f} Hz")
        return f
    except Exception:
        print("[warn] fs not parseable → default 250.0 Hz")
        return 250.0

X  = _resolve_field()
fs = _resolve_fs()

CONS_PATH = "cnt_flow_exports/consensus_edges.csv"
if not os.path.exists(CONS_PATH):
    # create a minimal consensus file from a fast pass so this never blocks
    print(f"[warn] {CONS_PATH} not found → making a quick provisional consensus.")
    # quick 2-band pass to seed edges
    from scipy import signal
    def band_pass(X, fs, f1, f2, order=4):
        b,a = signal.butter(order, [f1/(fs/2), f2/(fs/2)], btype='band')
        return signal.filtfilt(b,a,X,axis=0)
    def granger_xy(x, y, max_lag=8):
        Z = np.vstack([x,y]).T - np.mean(np.vstack([x,y]).T, axis=0)
        best, best_aic = (1.0,1.0), np.inf
        for p in range(1, max_lag+1):
            try:
                m = VAR(Z).fit(p)
                if m.aic < best_aic:
                    best_aic = m.aic
                    fx = m.test_causality(0,[1],kind='f')
                    fy = m.test_causality(1,[0],kind='f')
                    best = (fx.pvalue, fy.pvalue)
            except Exception:
                continue
        return best
    labels = [f"ch{j}" for j in range(X.shape[1])]
    fast_edges = []
    for (lab,f1,f2) in [("α",8,12), ("γ",30,45)]:
        Xb = band_pass(X, fs, f1, f2)
        for i in range(X.shape[1]):
            for j in range(X.shape[1]):
                if i==j: continue
                pyx, pxy = granger_xy(Xb[:,i], Xb[:,j], 8)
                q = multipletests([pyx, pxy], alpha=0.05, method='fdr_bh')[1][0]
                if q < 0.10:
                    fast_edges.append({"src":labels[i],"dst":labels[j],
                                       "bands_present":1,"mean_score":1.0,"bands_list":lab})
    cons = pd.DataFrame(fast_edges).drop_duplicates(subset=["src","dst"])
    os.makedirs("cnt_flow_exports", exist_ok=True)
    cons.to_csv(CONS_PATH, index=False)
    print(f"[ok] Wrote provisional file with {len(cons)} edges → {CONS_PATH}")
else:
    cons = pd.read_csv(CONS_PATH)
    print(f"[ok] Loaded consensus edges: {len(cons)} rows from {CONS_PATH}")

# ---- Quick robustness (downsized so it always prints fast) ----
from scipy import signal
def band_pass(X, fs, f1, f2, order=4):
    b,a = signal.butter(order, [f1/(fs/2), f2/(fs/2)], btype='band')
    return signal.filtfilt(b,a,X,axis=0)
def granger_xy(x, y, max_lag=8):
    Z = np.vstack([x, y]).T - np.mean(np.vstack([x, y]).T, axis=0)
    best_aic, best = np.inf, (1.0, 1.0)
    for p in range(1, max_lag+1):
        try:
            m = VAR(Z).fit(p)
            if m.aic < best_aic:
                best_aic = m.aic
                fx = m.test_causality(0,[1],kind='f')
                fy = m.test_causality(1,[0],kind='f')
                best = (fx.pvalue, fy.pvalue)
        except Exception:
            continue
    return best
def edge_q(Xb, s, d):  # forward direction q
    pyx, pxy = granger_xy(Xb[:,s], Xb[:,d], max_lag=8)
    return float(multipletests([pyx, pxy], alpha=0.05, method="fdr_bh")[1][0])

labels = [f"ch{j}" for j in range(X.shape[1])]
idx = {lab:i for i,lab in enumerate(labels)}
bands = [("α",8,12), ("γ",30,45)]  # quick scan
BLOCK = max(40, X.shape[0]//12)

rows=[]
for _, r in cons.iterrows():
    s = idx.get(r["src"]); d = idx.get(r["dst"])
    if s is None or d is None: continue
    hits = []
    for (lab,f1,f2) in bands:
        Xb = band_pass(X, fs, f1, f2)
        T  = Xb.shape[0]
        A, B = Xb[:T//2], Xb[T//2:]
        qA = edge_q(A, s, d); qB = edge_q(B, s, d)
        # time-reverse
        qF = edge_q(Xb, s, d)
        qR = edge_q(Xb[::-1].copy(), s, d)
        # block bootstrap (light)
        keep=0
        rng=np.random.default_rng(0)
        for _ in range(60):
            starts = rng.integers(0, T-BLOCK, size=max(1, T//BLOCK))
            Xbb = np.concatenate([Xb[st:st+BLOCK] for st in starts], axis=0)
            keep += (edge_q(Xbb, s, d) < 0.10)
        bb = (keep/60.0)*100
        # lag jitter (light)
        keep=0
        for _ in range(40):
            jit = np.random.randint(-2,3)
            Xj = Xb.copy(); Xj[:,d] = np.roll(Xj[:,d], jit)
            keep += (edge_q(Xj, s, d) < 0.10)
        lj = (keep/40.0)*100

        split = ( (qA<0.10) + (qB<0.10) )/2*100
        tr    = 100.0 if (qF<0.10 and qR>0.10) else 0.0
        R = 0.35*split + 0.25*tr + 0.25*bb + 0.15*lj
        rows.append({"edge": f"{r['src']}→{r['dst']}", "band": lab,
                     "split%": split, "time_rev%": tr, "boot%": bb, "lagjit%": lj, "robust%": R})

rob = pd.DataFrame(rows).sort_values(["robust%","edge"], ascending=[False,True]).reset_index(drop=True)
print("\n=== Quick robustness (2 bands) — top 10 band-edges ===")
print(rob.head(10).to_string(index=False))

# Aggregate per edge (mean across bands)
agg = (rob.groupby("edge", as_index=False)
          .agg(robust_score=("robust%","mean"),
               split=("split%","mean"),
               time_rev=("time_rev%","mean"),
               boot=("boot%","mean"),
               lagjit=("lagjit%","mean"),
               bands=("band", "nunique")))
agg = agg.sort_values(["robust_score","bands"], ascending=[False,False]).reset_index(drop=True)
print("\n=== Aggregated robustness per edge (mean across bands) — top 10 ===")
print(agg.head(10).to_string(index=False))


In [3]:
# CNT Consensus Causality (CCC) vs. Broadband Granger — Single Mega-Cell
# Telos x Aetheron — 2025-10-06
# Dependencies: numpy, scipy, matplotlib (no statsmodels needed). Tries to pip if missing.

import sys, math, time, random, importlib, subprocess
def _ensure(pkg):
    try:
        importlib.import_module(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--quiet"])
for p in ["numpy","scipy","matplotlib"]:
    _ensure(p)

import numpy as np
from numpy.linalg import lstsq
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=(7.5,5.0); plt.rcParams["figure.dpi"]=120

# -----------------------------
# 1) Synthetic truth generator
# -----------------------------
def kuramoto_network(T=12.0, dt=0.01, N=12, K=0.9, sparsity=0.75, noise=0.25, seed=7):
    """
    Coupled phase oscillators with directed, weighted adjacency (ground truth).
    Returns: phases [Tsteps,N], signals [Tsteps,N], true_adj (NxN, boolean, directed)
    """
    rng = np.random.default_rng(seed)
    tsteps = int(T/dt)
    # Directed adjacency with controlled density; no self-loops
    A = rng.random((N,N))
    A = (A > sparsity).astype(float)
    np.fill_diagonal(A, 0.0)
    # Weight edges (small heterogeneity)
    W = A * (0.6 + 0.8*rng.random((N,N)))
    # Natural frequencies
    w = rng.normal(1.5, 0.2, size=N)
    theta = rng.uniform(0, 2*np.pi, size=N)
    TH = np.zeros((tsteps, N))
    for t in range(tsteps):
        # Kuramoto with directed influence
        sin_terms = np.zeros(N)
        for i in range(N):
            # incoming influences to i from j
            diffs = theta - theta[i]
            sin_terms[i] = np.sum(W[i,:] * np.sin(diffs))
        dtheta = w + K * sin_terms + rng.normal(0, noise, size=N)
        theta = (theta + dt * dtheta) % (2*np.pi)
        TH[t,:] = theta
    # Observed signals: band-limited sinusoids + mild amplitude noise
    sig = np.sin(TH) + 0.15*rng.normal(0,1,(tsteps,N))
    return TH, sig, (W>0).astype(bool)

# ------------------------------------
# 2) Filters & frequency sub-bands
# ------------------------------------
def bandpass(x, fs, lo, hi, order=4):
    nyq = fs/2.0
    lo = max(1e-6, lo/nyq); hi = min(0.999, hi/nyq)
    b,a = butter(order, [lo,hi], btype='bandpass')
    return filtfilt(b,a,x,axis=0)

def multi_band(signals, fs):
    # Classic EEG-ish bands (Hz), but we’re synthetic; still useful consensus diversity
    bands = {
        "δ": (0.5, 4.0),
        "θ": (4.0, 8.0),
        "α": (8.0, 13.0),
        "β": (13.0, 30.0),
        "γ": (30.0, 48.0)   # keep below Nyquist
    }
    out = {}
    for k,(lo,hi) in bands.items():
        try:
            out[k] = bandpass(signals, fs, lo, hi)
        except Exception:
            # If bandpass fails (e.g., too-low/fs), fall back to raw for that band
            out[k] = signals.copy()
    return out

# ----------------------------------------------------
# 3) Minimal Granger F-test (no statsmodels required)
# ----------------------------------------------------
def _lag_stack(X, maxlag):
    """
    Build design matrices for VAR-like regression:
    y_t (for each series) against lagged predictors of ALL series up to maxlag.
    Returns (Y, Z), where Y is stacked targets, Z stacked regressors, and index map for (i target).
    """
    T, N = X.shape
    rows = T - maxlag
    Y = np.zeros((rows*N, ))  # concatenated targets per variable
    Z = np.zeros((rows*N, N*maxlag + 1))  # intercept + all lag blocks
    row = 0
    target_idx = []
    for i in range(N):
        y = X[maxlag:, i]
        # build regressors
        reg = [np.ones(rows)]
        for lag in range(1, maxlag+1):
            reg.append(X[maxlag - lag: T - lag, :].T.reshape(N, rows))
        R = np.vstack(reg)  # shape: (1 + N*maxlag, rows)
        Y[row:row+rows] = y
        Z[row:row+rows, :] = R.T
        target_idx.append((row, row+rows))
        row += rows
    return Y, Z, target_idx

def granger_matrix_F(X, maxlag=6):
    """
    For each directed pair j -> i, test whether including x_j lag-block improves
    prediction of y_i over the reduced model (without x_j).
    Returns matrix of p-values (NxN) with NaN on diagonal.
    """
    T,N = X.shape
    Y, Z, tblocks = _lag_stack(X, maxlag)
    # Precompute OLS for the full design (all predictors)
    beta_full, _, _, _ = lstsq(Z, Y, rcond=None)
    resid_full = Y - Z @ beta_full
    RSS_full = []
    for i,(a,b) in enumerate(tblocks):
        RSS_full.append(np.sum(resid_full[a:b]**2))
    RSS_full = np.array(RSS_full)

    # For each j, build reduced model by removing its lag-columns
    pvals = np.full((N,N), np.nan)
    rows = (T - maxlag)
    p_full = Z.shape[1]
    for i in range(N):
        # indices for target i within concatenated rows
        a,b = tblocks[i]
        # Find columns belonging to j's lags
        for j in range(N):
            if i == j:
                continue
            keep_cols = [0]  # intercept
            for lag in range(1, maxlag+1):
                for jj in range(N):
                    col = 1 + (lag-1)*N + jj
                    if jj == j:
                        continue
                    keep_cols.append(col)
            Z_red = Z[a:b][:, keep_cols]
            y_i = Y[a:b]
            beta_red, _, _, _ = lstsq(Z_red, y_i, rcond=None)
            resid_red = y_i - Z_red @ beta_red
            RSS_red = np.sum(resid_red**2)

            # F-statistic (nested): ( (RSS_red - RSS_full) / df_num ) / ( RSS_full / df_den )
            df_num = Z.shape[1] - len(keep_cols)     # number of parameters dropped = N lags of j
            df_den = rows - Z.shape[1]
            if df_den <= 0 or df_num <= 0:
                pvals[i,j] = 1.0
                continue
            F = ((RSS_red - RSS_full[i]) / df_num) / (RSS_full[i] / df_den)
            # Convert to p-value using survival function of F(df_num, df_den)
            from scipy.stats import f
            p = 1.0 - f.cdf(F, df_num, df_den)
            pvals[i,j] = max(min(p,1.0), 0.0)
    return pvals

# ---------------------------------------------
# 4) CCC: bands × lag-jitter × block-bootstrap
# ---------------------------------------------
def fdr_bh(pvals, alpha=0.05):
    """Benjamini–Hochberg across flattened p-values (ignoring NaNs). Returns boolean mask of rejections in the same shape."""
    P = pvals.copy()
    shape = P.shape
    pv = P[~np.isnan(P)].ravel()
    m = len(pv)
    if m==0: return np.zeros_like(P, dtype=bool)
    order = np.argsort(pv)
    ranked = np.empty_like(order); ranked[order] = np.arange(1, m+1)
    thresh = (ranked / m) * alpha
    reject_flat = pv <= thresh
    # ensure monotonicity
    if np.any(reject_flat):
        k = np.max(np.where(reject_flat)[0])
        cutoff = pv[order][k]
        rej = (P <= cutoff)
    else:
        rej = np.zeros_like(P, dtype=bool)
    rej[np.isnan(P)] = False
    return rej

def block_bootstrap_scores(X, fs, maxlag=6, B=120, block=None, jit=2, bands=None, rng=None):
    """
    Compute consensus scores across bands with lag jitter + block bootstrap.
    Returns:
      score_matrix (NxN), pval_matrix (NxN) via bootstrap null, and per-band stability.
    """
    if rng is None: rng = np.random.default_rng(0)
    T,N = X.shape
    if block is None: block = max(40, T//10)
    # Prepare bands
    if bands is None:
        bands = multi_band(X, fs)
    keys = list(bands.keys())

    # helper to compute -log10 p for a given band/lag choice
    def band_trial(bX, lag):
        P = granger_matrix_F(bX, maxlag=lag)
        with np.errstate(divide='ignore'):
            S = -np.log10(np.maximum(P, 1e-300))
        np.fill_diagonal(S, 0.0)
        return S, P

    # Original full-sample consensus with jit
    S_bag = []
    P_bag = []
    for _ in range(jit):
        lag = max(2, maxlag + rng.integers(-2, 3))  # jitter lag by ±2
        Stot = np.zeros((N,N)); Ptot = np.zeros((N,N))
        for k in keys:
            S,P = band_trial(bands[k], lag)
            Stot += S; Ptot += P
        S_bag.append(Stot/len(keys)); P_bag.append(Ptot/len(keys))
    S_full = np.mean(np.stack(S_bag,0),0)
    P_full = np.mean(np.stack(P_bag,0),0)

    # Block bootstrap to assess stability & null
    def block_resample(Xin):
        T,_ = Xin.shape
        idx = []
        while len(idx) < T:
            start = rng.integers(0, max(1, T-block))
            idx.extend(list(range(start, min(T, start+block))))
        idx = np.array(idx[:T])
        return Xin[idx,:]

    boots = []
    for b in range(B):
        # resample time within each band identically to preserve cross-band alignment
        idx = None
        bband = {}
        T,_ = X.shape
        # create a single idx per bootstrap
        idx = []
        while len(idx) < T:
            start = rng.integers(0, max(1, T-block))
            idx.extend(list(range(start, min(T, start+block))))
        idx = np.array(idx[:T])
        for k in keys:
            bband[k] = bands[k][idx,:]
        lag = max(2, maxlag + rng.integers(-2, 3))
        Stot = np.zeros((N,N))
        for k in keys:
            S,_ = band_trial(bband[k], lag)
            Stot += S
        boots.append(Stot/len(keys))
    boots = np.stack(boots,0)
    S_mu = boots.mean(0)
    S_sd = boots.std(0) + 1e-9

    # Z-score stability boost; and bootstrap p via upper-tail
    Z = (S_full - S_mu) / S_sd
    p_boot = 1.0 - (np.sum(boots >= S_full[None,:,:], axis=0) / boots.shape[0])
    np.fill_diagonal(p_boot, 1.0)

    # Final CCC score combines magnitude & stability
    score = np.maximum(0.0, S_full) * np.maximum(0.0, Z)
    return score, p_boot, S_full

# ---------------------------------------
# 5) Evaluation vs Broadband Granger
# ---------------------------------------
def evaluate(adj_true, rej_mask):
    """Precision, Recall, F1, AUCPR estimate via threshold sweep on score or pvals."""
    N = adj_true.shape[0]
    tp = np.sum(rej_mask & adj_true)
    fp = np.sum(rej_mask & (~adj_true) & (~np.eye(N,dtype=bool)))
    fn = np.sum((~rej_mask) & adj_true)
    precision = tp / max(1, tp+fp)
    recall    = tp / max(1, tp+fn)
    f1 = 2*precision*recall / max(1e-12, precision+recall)
    return precision, recall, f1

def auc_pr_from_scores(adj_true, scores, n_thresh=64):
    # sweep thresholds over quantiles of scores (excluding diag)
    N = scores.shape[0]
    S = scores.copy()
    S[np.eye(N, dtype=bool)] = -np.inf
    vals = S[np.isfinite(S)]
    if vals.size == 0:
        return 0.0
    qs = np.quantile(vals, np.linspace(0.0, 1.0, n_thresh))
    PR = []
    for th in qs:
        rej = (S >= th)
        p,r,_ = evaluate(adj_true, rej)
        PR.append((p,r))
    # approximate AUC via Riemann sum over recall, interpolating precision monotonically
    PR = np.array(PR)
    # ensure nonincreasing precision w.r.t recall
    order = np.argsort(PR[:,1])
    R = PR[order,1]; P = PR[order,0]
    for i in range(len(P)-2, -1, -1):
        P[i] = max(P[i], P[i+1])
    auc = np.trapz(P, R)
    return float(auc)

# ---------------------------------------
# 6) Experiment runner
# ---------------------------------------
def run_experiment(
    runs=24,
    N=10, T=12.0, dt=0.01,
    fs=None,
    K=0.9, sparsity=0.78, noise=0.25,
    maxlag=6, B=120, block=None, jit=3,
    alpha=0.05, seed=1234
):
    rng = np.random.default_rng(seed)
    AUC_ccc, AUC_gr = [], []
    F1_ccc, F1_gr = [], []
    summaries = []

    for r in range(runs):
        s = int(rng.integers(0, 10_000_000))
        TH, X, adj = kuramoto_network(T=T, dt=dt, N=N, K=K, sparsity=sparsity, noise=noise, seed=s)
        Tsteps = X.shape[0]
        if fs is None:
            fs = 1.0/dt

        # CCC
        bands = multi_band(X, fs)
        score, p_boot, Sfull = block_bootstrap_scores(
            X, fs, maxlag=maxlag, B=B, block=block, jit=jit, bands=bands, rng=rng
        )
        rej_ccc = fdr_bh(p_boot, alpha=alpha)
        p_cc, r_cc, f_cc = evaluate(adj, rej_ccc)
        auc_cc = auc_pr_from_scores(adj, score)

        # Broadband Granger baseline
        P_gr = granger_matrix_F(X, maxlag=maxlag)
        rej_gr = fdr_bh(P_gr, alpha=alpha)
        p_gr, r_gr, f_gr = evaluate(adj, rej_gr)

        # AUCPR for Granger via -log10 p as a score
        with np.errstate(divide='ignore'):
            S_gr = -np.log10(np.maximum(P_gr, 1e-300))
        np.fill_diagonal(S_gr, -np.inf)
        auc_gr = auc_pr_from_scores(adj, S_gr)

        AUC_ccc.append(auc_cc); AUC_gr.append(auc_gr)
        F1_ccc.append(f_cc);    F1_gr.append(f_gr)

        summaries.append(dict(seed=s, auc_ccc=auc_cc, auc_gr=auc_gr, f1_ccc=f_cc, f1_gr=f_gr))

    AUC_ccc = np.array(AUC_ccc); AUC_gr = np.array(AUC_gr)
    F1_ccc  = np.array(F1_ccc);  F1_gr  = np.array(F1_gr)

    # Paired permutation test on AUC deltas
    delta = AUC_ccc - AUC_gr
    observed = np.mean(delta)
    nperm = 5000
    rng2 = np.random.default_rng(seed+1)
    signs = rng2.choice([-1,1], size=(nperm, len(delta)))
    perm_means = np.mean(signs * delta[None,:], axis=1)
    p_perm = np.mean(perm_means >= observed)

    result = {
        "AUC_ccc_mean": float(np.mean(AUC_ccc)),
        "AUC_gr_mean":  float(np.mean(AUC_gr)),
        "AUC_ccc_median": float(np.median(AUC_ccc)),
        "AUC_gr_median":  float(np.median(AUC_gr)),
        "AUC_delta_mean": float(observed),
        "delta>0_fraction": float(np.mean(delta>0)),
        "p_perm_one_sided": float(p_perm),
        "F1_ccc_mean": float(np.mean(F1_ccc)),
        "F1_gr_mean":  float(np.mean(F1_gr)),
        "runs": runs,
        "summaries": summaries
    }
    return result, (AUC_ccc, AUC_gr), (F1_ccc, F1_gr)

# ---------------------------------------
# 7) Run it
# ---------------------------------------
start = time.time()
cfg = dict(
    runs=28,          # more runs => stronger test (keep runtime sane)
    N=10,             # nodes
    T=12.0, dt=0.01,  # 1200 samples
    K=0.95,           # coupling strength (moderate)
    sparsity=0.80,    # 20% edges present
    noise=0.28,       # phase noise
    maxlag=6,
    B=150,            # bootstrap reps
    jit=3,
    alpha=0.05,
    seed=424242
)
res, aucs, f1s = run_experiment(**cfg)
elapsed = time.time()-start

# ---------------------------------------
# 8) Display results & tiny plots
# ---------------------------------------
AUC_ccc, AUC_gr = aucs
F1_ccc, F1_gr   = f1s

print("=== BIG CLAIM TEST: CCC vs Broadband Granger ===")
print(f"Runs: {res['runs']}   Elapsed: {elapsed:.1f}s")
print(f"AUC(PR)  — CCC: mean {res['AUC_ccc_mean']:.3f} | Granger: mean {res['AUC_gr_mean']:.3f} "
      f"| Δ(mean): {res['AUC_delta_mean']:.3f}")
print(f"F1       — CCC: mean {res['F1_ccc_mean']:.3f} | Granger: mean {res['F1_gr_mean']:.3f}")
print(f"Fraction of runs where CCC AUC > Granger AUC: {res['delta>0_fraction']:.2f}")
print(f"Paired permutation p (one-sided, CCC>Granger): p = {res['p_perm_one_sided']:.5f}")

claim_ok = (res['delta>0_fraction'] >= 0.70) and (res['p_perm_one_sided'] < 0.01)
print("\nDECISION:", "✅ SUPPORTS CLAIM" if claim_ok else "❌ FAILS TO SUPPORT CLAIM")

# Simple visualization
fig, ax = plt.subplots()
ax.scatter(range(len(AUC_ccc)), AUC_ccc, label="CCC AUC(PR)", s=20)
ax.scatter(range(len(AUC_gr)),  AUC_gr,  label="Granger AUC(PR)", marker='x', s=24)
ax.set_xlabel("Run")
ax.set_ylabel("AUC(PR)")
ax.set_title("CCC vs Granger across runs")
ax.legend()
plt.tight_layout()
plt.show()


KeyboardInterrupt: 